In [1]:
import tensorflow as tf
print(tf.__version__)

2025-03-14 11:37:07.275546: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-14 11:37:10.055776: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.13.0


In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Input, mixed_precision
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout, BatchNormalization, Activation, Add, Multiply, Concatenate

import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, map_coordinates, median_filter, sobel, binary_dilation, distance_transform_edt
from sklearn.linear_model import RANSACRegressor
import os
import random
import time
from collections import defaultdict

# ✅ TensorFlow GPU 활성화 (변경 없음)
# GPU 메모리 사용량 제한
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # 두 설정 중 하나만 선택해야 합니다
        # 옵션 1: 메모리 성장 설정
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU 메모리 성장 설정 완료!")
        
        # 옵션 2: 메모리 제한 설정 (위 설정과 함께 사용 불가)
        # for gpu in gpus:
        #     tf.config.experimental.set_virtual_device_configuration(
        #         gpu,
        #         [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        #     )
        # print("✅ GPU 메모리 제한 설정 완료!")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ GPU를 사용할 수 없습니다. CPU 모드로 실행됩니다.")

# ✅ 학습 결과 저장 디렉토리 생성 (변경 없음)
if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

# ✅ 시각화 결과 저장 디렉토리 추가
if not os.path.exists('visualizations'):
    os.makedirs('visualizations')

# ✅ 코드 맨 앞에 추가 (변경 없음)
tf.config.run_functions_eagerly(False)


def convert_mde_image_to_depth(mde_image_path):
    """
    MDE 시각화 이미지를 깊이 정보로 변환
    
    Parameters:
    - mde_image_path: MDE 시각화 이미지 경로
    
    Returns:
    - 깊이 정보 배열 (float32)
    """
    # 그레이스케일로 이미지 로드
    mde_vis = cv2.imread(mde_image_path, cv2.IMREAD_GRAYSCALE)
    
    if mde_vis is None:
        print(f"❌ 이미지를 로드할 수 없습니다: {mde_image_path}")
        return None
    
    # 0-255 범위의 그레이스케일 이미지를 깊이 값으로 정규화
    # 예: 0-255 값을 0-10m(미터) 범위로 변환
    depth = mde_vis.astype(np.float32) / 255.0 * 10000.0
    
    return depth

def load_paired_data_from_existing_files(rgb_dir, depth_array_dir, depth_image_dir, mde_dir, max_samples):
    # 파일 목록 생성
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.startswith('color_') and f.endswith('.png')])
    depth_array_files = sorted([f for f in os.listdir(depth_array_dir) if f.startswith('depth_') and f.endswith('.npy')])
    depth_image_files = sorted([f for f in os.listdir(depth_image_dir) if f.startswith('depth_') and f.endswith('.png')])
    
    mde_npy_files = sorted([f for f in os.listdir(mde_dir) if f.endswith('_depth.npy')])
    mde_png_files = sorted([f for f in os.listdir(mde_dir) if f.endswith('_depth_vis.png')])

    print(f"📊 Found files: RGB {len(rgb_files)}, Depth Array {len(depth_array_files)}, Depth Image {len(depth_image_files)}")
    print(f"MDE NPY {len(mde_npy_files)}, MDE PNG {len(mde_png_files)}")

    # 타임스탬프 딕셔너리 생성
    timestamp_dict = {}
    
    # RGB 파일 처리
    for rgb_file in rgb_files:
        try:
            # 파일명에서 타임스탬프 추출 (color_20250314_154240.png)
            timestamp = rgb_file.replace('color_', '').replace('.png', '')
            
            timestamp_dict[timestamp] = {
                'rgb_file': rgb_file,
                'has_rgb': True,
                'depth_array_file': f'depth_{timestamp}.npy',
                'depth_image_file': f'depth_{timestamp}.png',
                'mde_npy_file': f'color_{timestamp}_depth.npy',
                'mde_png_file': f'color_{timestamp}_depth_vis.png'
            }
        except Exception as e:
            print(f"⚠️ RGB 파일 처리 중 오류: {rgb_file}, {e}")
    
    # 깊이 파일 처리
    for depth_file in depth_array_files + depth_image_files:
        try:
            # 파일명에서 타임스탬프 추출 (depth_20250314_154240.npy 또는 depth_20250314_154240.png)
            timestamp = depth_file.replace('depth_', '').replace('.npy', '').replace('.png', '')
            
            if timestamp in timestamp_dict:
                timestamp_dict[timestamp]['depth_valid'] = True
        except Exception as e:
            print(f"⚠️ 깊이 파일 처리 중 오류: {depth_file}, {e}")
    
    # MDE 파일 처리
    for mde_file in mde_npy_files + mde_png_files:
        try:
            # 파일명에서 타임스탬프 추출 (color_20250314_154240_depth.npy 또는 color_20250314_154240_depth_vis.png)
            timestamp = mde_file.split('_depth')[0].replace('color_', '')
            
            if timestamp in timestamp_dict:
                timestamp_dict[timestamp]['mde_valid'] = True
        except Exception as e:
            print(f"⚠️ MDE 파일 처리 중 오류: {mde_file}, {e}")
    
    # 유효한 타임스탬프 수집
    valid_timestamps = [
        ts for ts, info in timestamp_dict.items() 
        if info.get('has_rgb', False) and 
           info.get('depth_valid', False) and 
           info.get('mde_valid', False)
    ]
    
    print(f"✅ 매칭된 타임스탬프: {len(valid_timestamps)}")
    
    # 샘플 수 제한
    valid_timestamps = valid_timestamps[:max_samples]
    
    # 데이터 저장 리스트
    rgb_images = []
    depth_maps = []
    mde_maps = []
    
    # 매칭된 타임스탬프로 데이터 로드
    for timestamp in valid_timestamps:
        try:
            info = timestamp_dict[timestamp]
            
            # RGB 이미지 로드
            rgb_path = os.path.join(rgb_dir, info['rgb_file'])
            rgb = cv2.imread(rgb_path)
            if rgb is None:
                print(f"❌ RGB 이미지 로드 실패: {rgb_path}")
                continue
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
            
            # 깊이 맵 로드 (NPY 파일 우선)
            depth_npy_path = os.path.join(depth_array_dir, f'depth_{timestamp}.npy')
            depth_png_path = os.path.join(depth_image_dir, f'depth_{timestamp}.png')
            
            if os.path.exists(depth_npy_path):
                depth = np.load(depth_npy_path)
            else:
                depth = cv2.imread(depth_png_path, cv2.IMREAD_ANYDEPTH)
            
            # MDE 데이터 로드 (NPY 파일 우선)
            mde_npy_path = os.path.join(mde_dir, f'color_{timestamp}_depth.npy')
            
            if os.path.exists(mde_npy_path):
                mde = np.load(mde_npy_path)
            else:
                # PNG 파일을 깊이로 변환
                mde_png_path = os.path.join(mde_dir, f'color_{timestamp}_depth_vis.png')
                mde = convert_mde_image_to_depth(mde_png_path)
            
            # 데이터 추가
            rgb_images.append(rgb)
            depth_maps.append(depth)
            mde_maps.append(mde)
            
        except Exception as e:
            print(f"❌ 타임스탬프 {timestamp} 처리 중 오류: {e}")
            import traceback
            traceback.print_exc()
    
    print(f"✅ 총 로드된 샘플: {len(rgb_images)}")
    
    # NumPy 배열로 변환
    if len(rgb_images) > 0:
        return (
            np.array(rgb_images), 
            np.array(depth_maps), 
            np.array(mde_maps)
        )
    else:
        print("❌ 유효한 샘플 없음!")
        return None

def safe_load_image(path, color_conversion=cv2.COLOR_BGR2RGB):
    """안전한 이미지 로딩"""
    try:
        img = cv2.imread(path)
        if img is None:
            print(f"❌ 이미지 로드 실패: {path}")
            return None
        return cv2.cvtColor(img, color_conversion)
    except Exception as e:
        print(f"❌ 이미지 로드 중 오류: {path}, {e}")
        return None

def safe_load_depth(path, is_npy=False):
    """안전한 깊이 데이터 로딩"""
    try:
        if is_npy:
            depth = np.load(path)
        else:
            depth = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
        
        if depth is None:
            print(f"❌ 깊이 데이터 로드 실패: {path}")
            return None
        return depth
    except Exception as e:
        print(f"❌ 깊이 데이터 로드 중 오류: {path}, {e}")
        return None

def extract_timestamp_safely(filename):
    """안전한 타임스탬프 추출"""
    try:
        # color_20250314_154240.png 형식 가정
        parts = filename.replace('color_', '').replace('.png', '').split('_')
        if len(parts) >= 2:
            return f"{parts[0]}_{parts[1]}"
        else:
            print(f"❌ 타임스탬프 추출 실패: {filename}")
            return None
    except Exception as e:
        print(f"❌ 타임스탬프 추출 중 오류: {filename}, {e}")
        return None

def robust_dataset_loading(rgb_dir, depth_array_dir, depth_image_dir, mde_dir, indices, target_size=(64, 64)):
    """개선된 데이터셋 로딩 함수"""
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.startswith('color_') and f.endswith('.png')])
    
    rgb_images = []
    depth_maps = []
    mde_maps = []
    
    for index in indices:
        if index >= len(rgb_files):
            print(f"❌ 인덱스 {index}는 파일 범위를 벗어났습니다.")
            continue
        
        rgb_filename = rgb_files[index]
        timestamp = extract_timestamp_safely(rgb_filename)
        
        if not timestamp:
            continue
        
        # RGB 이미지 로드
        rgb_path = os.path.join(rgb_dir, rgb_filename)
        rgb = safe_load_image(rgb_path)
        if rgb is None:
            continue
        
        # 깊이 맵 로드 (우선순위: NPY > PNG)
        depth_npy_path = os.path.join(depth_array_dir, f'depth_{timestamp}.npy')
        depth_png_path = os.path.join(depth_image_dir, f'depth_{timestamp}.png')
        
        depth = safe_load_depth(depth_npy_path, is_npy=True)
        if depth is None:
            depth = safe_load_depth(depth_png_path)
        
        if depth is None:
            continue
        
        # MDE 맵 로드
        mde_path = os.path.join(mde_dir, f'color_{timestamp}_depth.npy')
        mde = safe_load_depth(mde_path, is_npy=True)
        
        if mde is None:
            continue
        
        # 크기 조정
        rgb_resized = cv2.resize(rgb, target_size, interpolation=cv2.INTER_AREA)
        depth_resized = cv2.resize(depth, target_size, interpolation=cv2.INTER_NEAREST)
        mde_resized = cv2.resize(mde, target_size, interpolation=cv2.INTER_NEAREST)
        
        rgb_images.append(rgb_resized)
        depth_maps.append(depth_resized)
        mde_maps.append(mde_resized)
    
    if not rgb_images:
        print("❌ 유효한 샘플이 없습니다!")
        return None
    
    return (
        np.array(rgb_images), 
        np.array(depth_maps), 
        np.array(mde_maps)
    )


# 데이터셋 생성 함수 - 기존 데이터 파일 활용
def create_dataset_from_existing_data(rgb_dir, depth_array_dir ,depth_image_dir, mde_dir, batch_size=8, target_size=(64, 64), max_samples=1000):
    """
    기존 데이터 파일에서 MDE 인식 모델을 위한 데이터셋 생성
    """
    # 매칭된 데이터 로드
    rgb_images, depth_maps, mde_maps = load_paired_data_from_existing_files(
        rgb_dir, depth_array_dir ,depth_image_dir, mde_dir, max_samples
    )
    
    print(f"로드된 데이터 형태: RGB {rgb_images.shape}, Depth {depth_maps.shape}, MDE {mde_maps.shape}")
    
    # 데이터 분할
    from sklearn.model_selection import train_test_split
    rgb_train, rgb_val, depth_train, depth_val, mde_train, mde_val = train_test_split(
        rgb_images, depth_maps, mde_maps, test_size=0.2, random_state=42
    )
    
    print(f"학습 데이터: {len(rgb_train)}개, 검증 데이터: {len(rgb_val)}개")
    
    # 기존 코드 유지
    train_dataset = process_and_create_dataset(
        rgb_train, depth_train, mde_train, batch_size, target_size
    )
    
    val_dataset = process_and_create_dataset(
        rgb_val, depth_val, mde_val, batch_size, target_size
    )
    
    return train_dataset, val_dataset

def process_and_create_dataset(rgb_images, depth_maps, mde_maps, batch_size, target_size):
    """
    배열에서 데이터셋 생성 및 전처리
    """
    inputs = []
    targets = []
    
    for i in range(len(rgb_images)):
        # RGB 이미지 크기 조정
        rgb = cv2.resize(rgb_images[i], target_size, interpolation=cv2.INTER_AREA)
        
        # 깊이 맵 크기 조정
        depth = cv2.resize(depth_maps[i], target_size, interpolation=cv2.INTER_NEAREST)
        
        # MDE 맵 크기 조정
        mde = cv2.resize(mde_maps[i], target_size, interpolation=cv2.INTER_NEAREST)
        
        # RGB 정규화
        rgb_norm = rgb.astype(np.float32) / 255.0
        
        # 신뢰도 맵 계산
        sensor_confidence = np.ones_like(depth, dtype=np.float32) * 0.6
        mde_confidence = np.ones_like(mde, dtype=np.float32) * 0.7
        
        # 입력 결합 - 7채널로 확장
        combined_input = np.concatenate([
            rgb_norm,
            np.expand_dims(depth, axis=-1),
            np.expand_dims(mde, axis=-1),  # MDE 깊이 추가
            np.expand_dims(sensor_confidence, axis=-1),
            np.expand_dims(mde_confidence, axis=-1)  # MDE 신뢰도 맵 추가
        ], axis=-1)
        
        # 타겟 깊이 (실측 깊이)
        target_depth = np.expand_dims(depth, axis=-1)
        
        inputs.append(combined_input)
        targets.append(target_depth)
    
    # 데이터 유효성 확인
    if len(inputs) == 0:
        print("처리된 입력 데이터가 없습니다!")
        return None
        
    # NumPy 배열로 변환
    inputs = np.array(inputs)
    targets = np.array(targets)
    
    print(f"입력 데이터 형태: {inputs.shape}")
    print(f"타겟 데이터 형태: {targets.shape}")
    
    # TensorFlow 데이터셋 생성
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    
    # 배치 및 프리페치
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset
    
# 신뢰도 맵 생성 함수 개선
def create_confidence_map(depth):
    """더 현실적인 신뢰도 맵 생성"""
    confidence = np.zeros_like(depth, dtype=np.float32)
    
    # 깊이 값이 존재하는 곳에는 높은 신뢰도 (하지만 1.0은 아님)
    valid_mask = (depth > 0)
    confidence[valid_mask] = 0.9
    
    # 에지에는 더 낮은 신뢰도 (에지 검출)
    if np.sum(valid_mask) > 100:  # 충분한 유효 픽셀이 있을 때만
        try:
            from scipy.ndimage import sobel
            edges = np.sqrt(sobel(depth, axis=0)**2 + sobel(depth, axis=1)**2)
            edge_mask = edges > np.percentile(edges[valid_mask], 80)
            confidence[edge_mask & valid_mask] = 0.7
        except:
            pass  # 예외가 발생하면 에지 처리 생략
    
    # 깊이 값이 없는 곳에는 낮은 신뢰도
    confidence[~valid_mask] = 0.3
    
    # 신뢰도 맵 부드럽게 처리
    try:
        from scipy.ndimage import gaussian_filter
        confidence = gaussian_filter(confidence, sigma=1.0)
    except:
        pass  # 예외가 발생하면 필터링 생략
    
    return confidence
    
# MDE와 깊이 맵 스케일 정렬 함수 (개선됨)
def align_mde_with_depth_improved(mde, depth):
    """Improved MDE and depth map scale alignment with better robust estimation"""
    # Convert MDE from m to mm
    mde_mm = mde * 1000.0
    
    # Valid depth pixel mask
    valid_mask = (depth > 0)
    
    if np.sum(valid_mask) < 100:  # Need more valid pixels for reliable estimation
        return mde_mm
    
    # Extract depth values excluding outliers (use 10-90 percentiles for stricter filtering)
    valid_depth = depth[valid_mask]
    valid_mde = mde_mm[valid_mask]
    
    # Calculate depth valid range (stricter outlier removal)
    depth_q10 = np.percentile(valid_depth, 10)
    depth_q90 = np.percentile(valid_depth, 90)
    valid_range_mask = (valid_depth >= depth_q10) & (valid_depth <= depth_q90)
    
    if np.sum(valid_range_mask) < 100:
        # Fall back to a more aggressive median-based scaling
        median_ratio = np.median(valid_depth) / np.median(valid_mde)
        median_ratio = np.clip(median_ratio, 0.5, 2.0)  # Limit scaling range
        return mde_mm * median_ratio
    
    # Use only values within valid range
    filtered_depth = valid_depth[valid_range_mask]
    filtered_mde = valid_mde[valid_range_mask]
    
    # Use more robust scaling approach (multi-scale fitting)
    try:
        # Try multiple scale estimations and choose the best one
        scale_options = []
        
        # Option 1: Simple median ratio
        median_ratio = np.median(filtered_depth) / np.median(filtered_mde)
        median_ratio = np.clip(median_ratio, 0.5, 2.0)
        scale_options.append(median_ratio)
        
        # Option 2: RANSAC regression for robust scale estimation
        model = RANSACRegressor(min_samples=0.7, max_trials=200)
        model.fit(filtered_mde.reshape(-1, 1), filtered_depth.reshape(-1, 1))
        ransac_scale = model.estimator_.coef_[0][0]
        ransac_scale = np.clip(ransac_scale, 0.5, 2.0)
        scale_options.append(ransac_scale)
        
        # Option 3: Quartile-based ratio (more robust than median)
        q3_depth = np.percentile(filtered_depth, 75)
        q3_mde = np.percentile(filtered_mde, 75)
        q3_ratio = q3_depth / q3_mde if q3_mde > 0 else 1.0
        q3_ratio = np.clip(q3_ratio, 0.5, 2.0)
        scale_options.append(q3_ratio)
        
        # Choose the median of all scale options for better stability
        final_scale = np.median(scale_options)
        
        # Apply scaling
        aligned_mde = mde_mm * final_scale
        
    except Exception as e:
        print(f"Error during MDE alignment: {str(e)}")
        # Fall back to simple median scaling
        median_ratio = np.median(filtered_depth) / np.median(filtered_mde)
        median_ratio = np.clip(median_ratio, 0.5, 2.0)
        aligned_mde = mde_mm * median_ratio
    
    return aligned_mde

# 개선된 하이브리드 깊이 맵 생성 함수
def create_hybrid_depth_robust(rgb, depth, mde):
    """More robust hybrid depth map creation with better blending strategy"""
    # Align MDE scale with depth
    aligned_mde = align_mde_with_depth_improved(mde, depth)
    
    # Valid depth pixel mask
    valid_mask = (depth > 0)
    
    # Missing depth pixel mask
    missing_mask = (depth == 0)
    
    # Initialize hybrid depth map
    hybrid_depth = depth.copy().astype(np.float32)
    
    # Identify reliable MDE regions (with stricter filtering)
    mde_valid = (aligned_mde > 0)
    
    # Use hybrid approach for confidence-based blending
    if np.any(valid_mask) and np.any(mde_valid):
        # Calculate statistics for better outlier detection
        valid_depth_values = depth[valid_mask]
        
        # Calculate depth value valid range using more robust IQR method
        depth_q25 = np.percentile(valid_depth_values, 25)
        depth_q75 = np.percentile(valid_depth_values, 75)
        depth_iqr = depth_q75 - depth_q25
        depth_median = np.median(valid_depth_values)
        
        # Stricter bounds for outlier detection
        lower_bound = depth_median - 1.5 * depth_iqr
        upper_bound = depth_median + 1.5 * depth_iqr
        
        # MDE outlier mask
        mde_outlier = (aligned_mde < lower_bound) | (aligned_mde > upper_bound)
        
        # Use only valid and non-outlier MDE
        reliable_mde_mask = mde_valid & ~mde_outlier
        
        # Fill missing parts with reliable MDE
        fill_mask = missing_mask & reliable_mde_mask
        hybrid_depth[fill_mask] = aligned_mde[fill_mask]
        
        # Create confidence-based blending for partially valid regions
        # (areas where both depth and MDE exist, but we want to use the more reliable one)
        blend_mask = valid_mask & reliable_mde_mask
        
        # Edge detection on depth map for finding discontinuities
        depth_edges = np.sqrt(sobel(depth, axis=0)**2 + sobel(depth, axis=1)**2)
        edge_mask = depth_edges > np.percentile(depth_edges[depth > 0], 85)
        
        # Higher MDE confidence near edges (depth sensors often fail at edges)
        edge_dilated = binary_dilation(edge_mask, iterations=3)
        
        # Blend based on confidence - use MDE more near edges
        edge_blend_mask = blend_mask & edge_dilated
        if np.any(edge_blend_mask):
            # Smooth blending - 70% MDE, 30% depth at edges
            hybrid_depth[edge_blend_mask] = 0.7 * aligned_mde[edge_blend_mask] + 0.3 * depth[edge_blend_mask]
    
    # Apply median filtering to remove any remaining noise
    if np.sum(hybrid_depth > 0) > 100:
        # Only apply median filter to areas with depth values
        valid_hybrid = hybrid_depth > 0
        filtered = median_filter(hybrid_depth, size=3)
        hybrid_depth[valid_hybrid] = filtered[valid_hybrid]
    
    # Smooth confidence transitions
    confidence_map = create_confidence_map(depth)
    
    return hybrid_depth, confidence_map


def prepare_model_input(rgb, depth, mde, target_size=(192, 192)):
    """
    모델 입력 데이터를 준비하는 함수
    
    매개변수:
    - rgb: RGB 이미지
    - depth: 실측 깊이 맵
    - mde: MDE 깊이 맵
    - target_size: 모델 입력 크기 (기본값 192x192)
    
    반환값:
    - model_input: 모델 입력 데이터 (배치 차원 포함)
    - hybrid_depth: 생성된 하이브리드 깊이 맵
    - confidence_map: 신뢰도 맵
    """
    # 이미지 크기 조정
    rgb_resized = cv2.resize(rgb, target_size, interpolation=cv2.INTER_AREA)
    depth_resized = cv2.resize(depth, target_size, interpolation=cv2.INTER_NEAREST)
    mde_resized = cv2.resize(mde, target_size, interpolation=cv2.INTER_NEAREST)
    
    # 하이브리드 깊이 맵 생성
    hybrid_depth, _ = create_hybrid_depth_robust(rgb_resized, depth_resized, mde_resized)
    
    # 신뢰도 맵 생성 - 여기를 수정
    confidence_map = create_confidence_map(depth_resized)  # 새 함수 사용
    
    # 입력 전처리
    rgb_norm = rgb_resized.astype(np.float32) / 255.0  # RGB 정규화
    
    # 차원 확장
    hybrid_depth_expanded = np.expand_dims(hybrid_depth, axis=-1)
    confidence_map_expanded = np.expand_dims(confidence_map, axis=-1)
    
    # 입력 결합
    combined_input = np.concatenate([rgb_norm, hybrid_depth_expanded, confidence_map_expanded], axis=-1)
    
    # 배치 차원 추가
    model_input = np.expand_dims(combined_input, axis=0)
    
    return model_input, hybrid_depth, confidence_map

def apply_depth_correction(model, rgb, depth, mde, target_size=(192, 192)):
    """
    전체 깊이 보정 과정을 처리하는 함수
    
    매개변수:
    - model: 깊이 보정 모델
    - rgb: RGB 이미지
    - depth: 실측 깊이 맵
    - mde: MDE 깊이 맵
    - target_size: 모델 입력 크기 (기본값 192x192)
    
    반환값:
    - corrected_depth: 보정된 깊이 맵
    - hybrid_depth: 하이브리드 깊이 맵 (모델 입력으로 사용됨)
    - confidence_map: 신뢰도 맵
    """
    # 모델 입력 준비
    model_input, hybrid_depth, confidence_map = prepare_model_input(rgb, depth, mde, target_size)
    
    # 모델 예측
    prediction = model.predict(model_input)
    
    # 결과 추출 (배치 차원 제거 및 채널 차원 제거)
    corrected_depth = prediction[0, :, :, 0]
    
    # 원본 크기로 다시 확장
    corrected_depth = cv2.resize(corrected_depth, (depth.shape[1], depth.shape[0]), interpolation=cv2.INTER_NEAREST)
    hybrid_depth = cv2.resize(hybrid_depth, (depth.shape[1], depth.shape[0]), interpolation=cv2.INTER_NEAREST)
    confidence_map = cv2.resize(confidence_map, (depth.shape[1], depth.shape[0]), interpolation=cv2.INTER_NEAREST)
    
    return corrected_depth, hybrid_depth, confidence_map

# 3. 손실 함수 개선 - 에지 보존에 더 큰 가중치 부여
def improved_depth_correction_loss_v3(y_true, y_pred):
    """강화된 깊이 보정 손실 함수"""
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # 유효한 깊이 마스크
    mask = tf.cast(tf.greater(y_true, 0), tf.float32)
    mask_sum = tf.reduce_sum(mask)
    
    # 반드시 기본 손실 포함 (마스크가 비어 있어도 학습 가능하도록)
    base_loss = 0.0001 * tf.reduce_mean(tf.abs(y_true - y_pred))
    
    # 마스크 유효성 검사
    if_valid = lambda: tf.reduce_sum(tf.abs(y_true - y_pred) * mask) / tf.maximum(mask_sum, 1.0)
    if_invalid = lambda: base_loss
    
    # 유효한 마스크가 있으면 마스크된 손실 반환, 그렇지 않으면 기본 손실 반환
    l1_loss = tf.cond(tf.greater(mask_sum, 0), if_valid, if_invalid)
    
    # 에지 인식 그래디언트 손실 (원래 코드와 동일)
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)

    grad_true = tf.sqrt(tf.square(dy_true) + tf.square(dx_true) + 1e-8)
    grad_pred = tf.sqrt(tf.square(dy_pred) + tf.square(dx_pred) + 1e-8)

    grad_loss = tf.reduce_mean(tf.abs(grad_true - grad_pred))  # 간단한 평균으로 계산

    # 손실 조합 (기본 손실 포함)
    total_loss = l1_loss + 0.5 * grad_loss + base_loss
    
    return total_loss

class NanLossCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is None:
            logs = {}
        loss = logs.get('loss')
        if loss is not None and (np.isnan(loss) or np.isinf(loss)):
            print(f"NaN/Inf 손실 감지됨: {loss}, 학습 중지")
            self.model.stop_training = True
            
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        loss = logs.get('loss')
        if loss is not None and (np.isnan(loss) or np.isinf(loss)):
            print(f"에폭 {epoch}에서 NaN/Inf 손실 감지됨: {loss}, 학습 중지")
            self.model.stop_training = True

def ultra_stable_depth_loss_v2(y_true, y_pred):
    """
    개선된 깊이 손실 함수 - NaN 방지 및 안정성 강화
    
    주요 개선점:
    1. 극단적인 클리핑
    2. 소프트 마스킹
    3. 유효 깊이 픽셀에 대한 가중치 부여
    4. 다중 손실 항목
    """
    # 타입 캐스팅 및 안전한 변환
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # 엡실론 값 - 매우 작은 값으로 나누기/로그 연산 방지
    epsilon = 1e-7
    
    # 극단적인 클리핑 (0-1 범위, 아주 작은 마진 포함)
    y_true = tf.clip_by_value(y_true, epsilon, 1.0 - epsilon)
    y_pred = tf.clip_by_value(y_pred, epsilon, 1.0 - epsilon)
    
    # 유효한 깊이 픽셀 마스크 (0보다 큰 값)
    valid_mask = tf.cast(tf.greater(y_true, epsilon), tf.float32)
    
    # 마스크된 유효 픽셀 수
    mask_sum = tf.maximum(tf.reduce_sum(valid_mask), 1.0)
    
    # L1 손실 (절대 오차)
    l1_loss = tf.reduce_sum(tf.abs(y_true - y_pred) * valid_mask) / mask_sum
    
    # 로그 코시 손실 - 극단값에 더 강건함
    log_cosh_loss = tf.reduce_sum(
        tf.math.log(tf.cosh(y_true - y_pred)) * valid_mask
    ) / mask_sum
    
    # 그래디언트 손실 - 에지 보존
    dy_true, dx_true = tf.image.image_gradients(y_true)
    dy_pred, dx_pred = tf.image.image_gradients(y_pred)
    
    # 그래디언트 크기 계산
    grad_true = tf.sqrt(tf.square(dy_true) + tf.square(dx_true) + epsilon)
    grad_pred = tf.sqrt(tf.square(dy_pred) + tf.square(dx_pred) + epsilon)
    
    # 그래디언트 손실
    grad_loss = tf.reduce_mean(tf.abs(grad_true - grad_pred))
    
    # 가중치를 적용한 최종 손실
    total_loss = (
        1.0 * l1_loss +  # 주요 깊이 오차
        0.5 * log_cosh_loss +  # 극단값 강건성
        0.3 * grad_loss +  # 에지 보존
        0.1 * tf.reduce_mean(tf.square(y_true - y_pred))  # 추가 안정성
    )
    
    # NaN 방지를 위한 최종 안전장치
    return tf.where(tf.math.is_finite(total_loss), total_loss, epsilon)

def safe_clip_gradients(grads_and_vars, clip_norm=1.0):
    """
    그래디언트 클리핑 및 NaN 방지 함수
    
    매개변수:
    - grads_and_vars: 그래디언트와 변수 쌍
    - clip_norm: 최대 그래디언트 노름
    
    반환값:
    안전하게 클리핑된 그래디언트와 변수 쌍
    """
    clipped_grads_and_vars = []
    for grad, var in grads_and_vars:
        if grad is not None:
            # NaN 값 제거
            grad = tf.where(tf.math.is_finite(grad), grad, tf.zeros_like(grad))
            
            # 그래디언트 클리핑
            grad = tf.clip_by_norm(grad, clip_norm)
            
            clipped_grads_and_vars.append((grad, var))
        else:
            clipped_grads_and_vars.append((grad, var))
    
    return clipped_grads_and_vars

class SafeAdamOptimizer(tf.keras.optimizers.Adam):
    """
    NaN에 강건한 개선된 Adam 옵티마이저
    """
    def _compute_gradients(self, loss, var_list):
        # 원래 그래디언트 계산
        grads_and_vars = super()._compute_gradients(loss, var_list)
        
        # 안전한 그래디언트 클리핑 적용
        return safe_clip_gradients(grads_and_vars)

# ====================================================================================
# ✅✅✅ 3. 모델 아키텍처 단순화 - 더 단순하고 안정적인 U-Net 모델
# ====================================================================================

# 개선된 컨볼루션 블록 함수
def conv_block_improved(inputs, filters, kernel_size=3, dropout_rate=0.0, use_residual=True):
    """개선된 컨볼루션 블록 - 더 안정적인 학습을 위한 정규화 기능 강화"""
    # 첫 번째 컨볼루션
    x = layers.Conv2D(
        filters, 
        kernel_size=kernel_size, 
        padding="same",
        kernel_initializer=tf.keras.initializers.HeNormal(),  # He 초기화 사용
        kernel_regularizer=l2(1e-5)  # L2 정규화 추가
    )(inputs)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    if dropout_rate > 0:
        x = Dropout(dropout_rate)(x)
        
    # 두 번째 컨볼루션
    x = layers.Conv2D(
        filters, 
        kernel_size=kernel_size, 
        padding="same",
        kernel_initializer=tf.keras.initializers.HeNormal(),
        kernel_regularizer=l2(1e-5)  # L2 정규화 추가
    )(x)
    
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # 잔차 연결 (필요한 경우)
    if use_residual:
        # 채널 수 맞추기
        shortcut = inputs
        if int(inputs.shape[-1]) != filters:
            shortcut = layers.Conv2D(filters, (1, 1), padding="same")(inputs)
            shortcut = BatchNormalization()(shortcut)
        
        x = Add()([x, shortcut])
        
    return x

def resize_compatible_dimensions(rgb_images, depth_maps, mde_predictions, target_size=(60, 80)):
    """U-Net에 적합한 크기로 이미지 조정 (항상 짝수 차원)"""
    # 각 차원이 8의 배수가 되도록 조정 (3번의 MaxPooling을 고려)
    h, w = target_size
    h_adjusted = (h // 8) * 8
    w_adjusted = (w // 8) * 8
    
    if h_adjusted != h or w_adjusted != w:
        print(f"크기가 조정됨: {target_size} -> {(h_adjusted, w_adjusted)}")
    
    return resize_dataset(rgb_images, depth_maps, mde_predictions, (h_adjusted, w_adjusted))

# 개선된 어텐션 블록 - MDE 깊이 모델에 특화된 버전

def channel_attention_block(inputs, ratio=8):
    """
    채널 어텐션 블록 - 각 특징 채널의 중요도를 학습
    입력의 채널 수에 따라 중요도 가중치 반환
    """
    channel = inputs.shape[-1]
    
    # 전역 평균 풀링
    avg_pool = layers.GlobalAveragePooling2D()(inputs)
    avg_pool = layers.Reshape((1, 1, channel))(avg_pool)
    
    # 전역 최대 풀링
    max_pool = layers.GlobalMaxPooling2D()(inputs)
    max_pool = layers.Reshape((1, 1, channel))(max_pool)
    
    # 공유 MLP를 통한 채널 압축 및 확장
    shared_mlp_1 = layers.Dense(channel // ratio, activation='relu', 
                                kernel_initializer='he_normal', 
                                use_bias=True, 
                                bias_initializer='zeros')
    shared_mlp_2 = layers.Dense(channel, 
                                kernel_initializer='he_normal', 
                                use_bias=True, 
                                bias_initializer='zeros')
    
    # 평균 풀링 경로
    avg_pool = shared_mlp_1(avg_pool)
    avg_pool = shared_mlp_2(avg_pool)
    
    # 최대 풀링 경로
    max_pool = shared_mlp_1(max_pool)
    max_pool = shared_mlp_2(max_pool)
    
    # 특징 융합
    attention = layers.Add()([avg_pool, max_pool])
    attention = Activation('sigmoid')(attention)
    
    # 채널별 가중치 적용
    return layers.Multiply()([inputs, attention])

def spatial_attention_block(inputs):
    """
    공간 어텐션 블록 - 이미지의 어느 부분이 중요한지 학습
    """
    # 채널 축을 따라 평균 및 최대값 계산
    avg_pool = layers.Lambda(lambda x: K.mean(x, axis=-1, keepdims=True))(inputs)
    max_pool = layers.Lambda(lambda x: K.max(x, axis=-1, keepdims=True))(inputs)
    
    # 두 특징 연결
    concat = layers.Concatenate()([avg_pool, max_pool])
    
    # 공간 어텐션 맵 생성 - 큰 커널 사용으로 더 넓은 범위 고려
    attention = layers.Conv2D(1, kernel_size=7, strides=1, padding='same', 
                             activation='sigmoid', 
                             kernel_initializer='he_normal', 
                             use_bias=False)(concat)
    
    # 공간 가중치 적용
    return layers.Multiply()([inputs, attention])

def cbam_block(inputs, ratio=8):
    """
    CBAM(Convolutional Block Attention Module) - 채널과 공간 어텐션 결합
    """
    # 채널 어텐션 적용
    channel_attention = channel_attention_block(inputs, ratio)
    
    # 공간 어텐션 적용
    spatial_attention = spatial_attention_block(channel_attention)
    
    return spatial_attention

def depth_aware_attention_block(rgb_features, depth_features, mde_features):
    """
    깊이 인식 어텐션 블록 - RGB, 센서 깊이, MDE 특징 간의 상호작용 학습
    """
    # 채널 수 결정 및 1x1 컨볼루션으로 채널 정규화
    channels = max(rgb_features.shape[-1], depth_features.shape[-1], mde_features.shape[-1])
    
    # 채널 수 맞추기 위한 1x1 컨볼루션
    rgb_features = layers.Conv2D(channels, kernel_size=1, padding='same')(rgb_features)
    depth_features = layers.Conv2D(channels, kernel_size=1, padding='same')(depth_features)
    mde_features = layers.Conv2D(channels, kernel_size=1, padding='same')(mde_features)
    
    # 모든 특징 연결
    all_features = layers.Concatenate()([rgb_features, depth_features, mde_features])
    
    # 상관관계 학습 (각 특징 간의 상호작용 캡처)
    correlation = layers.Conv2D(channels, kernel_size=3, padding='same')(all_features)
    correlation = BatchNormalization()(correlation)
    correlation = Activation('relu')(correlation)
    
    # 각 특징별 가중치 생성
    weights = layers.Conv2D(3, kernel_size=1, padding='same', activation='softmax')(correlation)
    
    # 가중치 분리
    rgb_weight = layers.Lambda(lambda x: x[:, :, :, 0:1])(weights)
    depth_weight = layers.Lambda(lambda x: x[:, :, :, 1:2])(weights)
    mde_weight = layers.Lambda(lambda x: x[:, :, :, 2:3])(weights)
    
    # 가중치 확장
    rgb_weight = layers.Lambda(lambda x: K.repeat_elements(x, channels, axis=-1))(rgb_weight)
    depth_weight = layers.Lambda(lambda x: K.repeat_elements(x, channels, axis=-1))(depth_weight)
    mde_weight = layers.Lambda(lambda x: K.repeat_elements(x, channels, axis=-1))(mde_weight)
    
    # 가중 특징
    weighted_rgb = layers.Multiply()([rgb_features, rgb_weight])
    weighted_depth = layers.Multiply()([depth_features, depth_weight])
    weighted_mde = layers.Multiply()([mde_features, mde_weight])
    
    # 가중 특징 결합
    weighted_features = layers.Add()([weighted_rgb, weighted_depth, weighted_mde])
    
    # CBAM 어텐션 적용
    enhanced_features = cbam_block(weighted_features)
    
    # 채널 수 일치를 위해 1x1 컨볼루션 사용
    enhanced_features = layers.Conv2D(channels, kernel_size=1, padding='same')(enhanced_features)
    
    # 잔차 연결 - 1x1 컨볼루션으로 채널 및 차원 정규화
    residual_connection = layers.Conv2D(channels, kernel_size=1, padding='same')(all_features)
    
    # 잔차 연결
    output = layers.Add()([residual_connection, enhanced_features])
    
    return output

# 보정 어텐션 계층 - 모델의 어느 부분이 더 많은 보정을 필요로 하는지 예측
def correction_attention_block(rgb, depth, mde, confidence):
    """
    보정 어텐션 블록 - 어느 영역이 보정이 필요한지 예측
    """
    # 모든 입력 정보 결합
    combined = layers.Concatenate()([rgb, depth, mde, confidence])
    
    # 보정 필요 영역 예측 네트워크
    x = layers.Conv2D(16, kernel_size=3, padding='same')(combined)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    
    x = layers.Conv2D(8, kernel_size=3, padding='same')(x)
    x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
    x = Activation('relu')(x)
    
    # 보정 필요 맵 생성 (높은 값 = 많은 보정 필요)
    correction_map = layers.Conv2D(1, kernel_size=3, padding='same', activation='sigmoid')(x)
    
    # 에지와 불확실성 강조
    edge_detector = layers.Conv2D(1, kernel_size=3, padding='same', 
                                 kernel_initializer=tf.keras.initializers.Constant([
                                     [-1, -1, -1],
                                     [-1, 8, -1],
                                     [-1, -1, -1]
                                 ]))(depth)
    
    edge_response = layers.Lambda(lambda x: K.abs(x))(edge_detector)
    edge_response = layers.Lambda(lambda x: K.sigmoid(5 * x))(edge_response)
    
    # 에지와 보정 맵 결합
    final_attention = layers.Add()([correction_map, 0.5 * edge_response])
    final_attention = layers.Lambda(lambda x: K.clip(x, 0, 1))(final_attention)
    
    return final_attention

def create_mde_aware_correction_model_with_attention(input_shape=(64, 64, 7)):
    """
    어텐션 메커니즘이 강화된 MDE 인식 깊이 보정 모델
    
    입력:
    - RGB (3채널)
    - 센서 깊이 (1채널)
    - MDE 예측 깊이 (1채널)
    - 센서 깊이 신뢰도 (1채널)
    - MDE 예측 신뢰도 (1채널)
    """
    inputs = Input(input_shape)

    rgb = layers.Lambda(lambda x: x[:, :, :, :3])(inputs)
    sensor_depth = layers.Lambda(lambda x: x[:, :, :, 3:4])(inputs)
    mde_depth = layers.Lambda(lambda x: x[:, :, :, 4:5])(inputs)
    sensor_confidence = layers.Lambda(lambda x: x[:, :, :, 5:6])(inputs)
    mde_confidence = layers.Lambda(lambda x: x[:, :, :, 6:7])(inputs)
    
    # 보정 어텐션 계산 - 어느 부분이 보정이 필요한지 예측
    correction_attention = correction_attention_block(
        rgb, sensor_depth, mde_depth,
        layers.Concatenate()([sensor_confidence, mde_confidence])
    )
    
    #--------------------------------------------------------------------------
    # 인코더 부분 - 각 브랜치에서 특징 추출
    #--------------------------------------------------------------------------
    
    # RGB 인코더 브랜치
    rgb_conv1 = layers.Conv2D(32, (3, 3), padding='same')(rgb)
    rgb_conv1 = BatchNormalization()(rgb_conv1)
    rgb_conv1 = Activation('relu')(rgb_conv1)
    # 채널 어텐션 적용
    rgb_conv1 = channel_attention_block(rgb_conv1)
    rgb_pool1 = layers.MaxPooling2D((2, 2))(rgb_conv1)
    
    rgb_conv2 = layers.Conv2D(64, (3, 3), padding='same')(rgb_pool1)
    rgb_conv2 = BatchNormalization()(rgb_conv2)
    rgb_conv2 = Activation('relu')(rgb_conv2)
    # 채널 어텐션 적용
    rgb_conv2 = channel_attention_block(rgb_conv2)
    rgb_pool2 = layers.MaxPooling2D((2, 2))(rgb_conv2)
    
    # 깊이 인코더 브랜치 (센서 깊이 + 신뢰도)
    sensor_depth_with_conf = layers.Concatenate()([sensor_depth, sensor_confidence])
    depth_conv1 = layers.Conv2D(32, (3, 3), padding='same')(sensor_depth_with_conf)
    depth_conv1 = BatchNormalization()(depth_conv1)
    depth_conv1 = Activation('relu')(depth_conv1)
    # 공간 어텐션 적용
    depth_conv1 = spatial_attention_block(depth_conv1)
    depth_pool1 = layers.MaxPooling2D((2, 2))(depth_conv1)
    
    depth_conv2 = layers.Conv2D(64, (3, 3), padding='same')(depth_pool1)
    depth_conv2 = BatchNormalization()(depth_conv2)
    depth_conv2 = Activation('relu')(depth_conv2)
    # 공간 어텐션 적용
    depth_conv2 = spatial_attention_block(depth_conv2)
    depth_pool2 = layers.MaxPooling2D((2, 2))(depth_conv2)
    
    # MDE 인코더 브랜치 (MDE 예측 + 신뢰도)
    mde_with_conf = layers.Concatenate()([mde_depth, mde_confidence])
    mde_conv1 = layers.Conv2D(32, (3, 3), padding='same')(mde_with_conf)
    mde_conv1 = BatchNormalization()(mde_conv1)
    mde_conv1 = Activation('relu')(mde_conv1)
    # 공간 어텐션 적용
    mde_conv1 = spatial_attention_block(mde_conv1)
    mde_pool1 = layers.MaxPooling2D((2, 2))(mde_conv1)
    
    mde_conv2 = layers.Conv2D(64, (3, 3), padding='same')(mde_pool1)
    mde_conv2 = BatchNormalization()(mde_conv2)
    mde_conv2 = Activation('relu')(mde_conv2)
    # 공간 어텐션 적용
    mde_conv2 = spatial_attention_block(mde_conv2)
    mde_pool2 = layers.MaxPooling2D((2, 2))(mde_conv2)
    
    # 차이 학습 브랜치 - 깊이 센서와 MDE 간의 차이 패턴 학습
    depth_diff = layers.Subtract()([sensor_depth, mde_depth])
    diff_conv1 = layers.Conv2D(32, (3, 3), padding='same')(depth_diff)
    diff_conv1 = BatchNormalization()(diff_conv1)
    diff_conv1 = Activation('relu')(diff_conv1)
    diff_pool1 = layers.MaxPooling2D((2, 2))(diff_conv1)
    
    diff_conv2 = layers.Conv2D(64, (3, 3), padding='same')(diff_pool1)
    diff_conv2 = BatchNormalization()(diff_conv2)
    diff_conv2 = Activation('relu')(diff_conv2)
    diff_pool2 = layers.MaxPooling2D((2, 2))(diff_conv2)
    
    #--------------------------------------------------------------------------
    # 중간층 - 특징 융합 및 어텐션 적용
    #--------------------------------------------------------------------------
    
    # 첫 번째 레벨 어텐션 적용 특징
    level1_attention = depth_aware_attention_block(rgb_conv1, depth_conv1, mde_conv1)
    level1_pool = layers.MaxPooling2D((2, 2))(level1_attention)
    
    # 두 번째 레벨 어텐션 적용 특징
    level2_attention = depth_aware_attention_block(rgb_conv2, depth_conv2, mde_conv2)
    
    # 인코더 특징 융합
    encoder_fusion = layers.Concatenate()([rgb_pool2, depth_pool2, mde_pool2, diff_pool2])
    
    # CBAM 어텐션 적용
    attended_fusion = cbam_block(encoder_fusion)
    
    # 병목층
    bottleneck = layers.Conv2D(256, (3, 3), padding='same')(attended_fusion)
    bottleneck = BatchNormalization()(bottleneck)
    bottleneck = Activation('relu')(bottleneck)
    
    #--------------------------------------------------------------------------
    # 디코더 부분 - 업샘플링 및 스킵 연결
    #--------------------------------------------------------------------------
    
    # 첫 번째 업샘플링
    up1 = layers.UpSampling2D((2, 2))(bottleneck)
    up1 = layers.Conv2D(128, (3, 3), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Activation('relu')(up1)
    
    # 스킵 연결
    skip1 = layers.Concatenate()([up1, level2_attention])
    
    # 두 번째 업샘플링
    up2 = layers.UpSampling2D((2, 2))(skip1)
    up2 = layers.Conv2D(64, (3, 3), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Activation('relu')(up2)
    
    # 스킵 연결
    skip2 = layers.Concatenate()([up2, level1_attention])
    
    #--------------------------------------------------------------------------
    # 출력 부분 - 깊이 융합 및 보정
    #--------------------------------------------------------------------------
    
    # 최종 특징 계산
    final_features = layers.Conv2D(32, (3, 3), padding='same')(skip2)
    final_features = BatchNormalization()(final_features)
    final_features = Activation('relu')(final_features)

    # 보정 어텐션 적용
    attended_features = layers.Multiply()([
        final_features,
        layers.Lambda(lambda x: K.repeat_elements(x, 32, axis=-1))(correction_attention)
    ])

    # 깊이 잔차 예측
    correction = layers.Conv2D(1, (3, 3), padding='same')(attended_features)
    correction = layers.Lambda(lambda x: 0.1 * K.tanh(x))(correction)  # 보정 범위 축소 (0-1 범위 기준)

    # 신뢰도 기반 가중치 계산
    fusion_weight = layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')(final_features)

    # 센서 깊이와 MDE 깊이의 유효성 마스크
    sensor_valid = layers.Lambda(lambda x: K.cast(K.greater(x, 0), 'float32'))(sensor_depth)
    mde_valid = layers.Lambda(lambda x: K.cast(K.greater(x, 0), 'float32'))(mde_depth)

    # 입력 깊이 융합 (신뢰도 기반)
    base_depth = layers.Lambda(lambda x:
                              x[0] * x[1] * x[2] * x[3] +
                              x[4] * (1-x[1]) * x[3] +
                              x[4] * (1-x[3])
                             )([
        sensor_depth,
        fusion_weight,
        sensor_confidence,
        sensor_valid,
        mde_depth
    ])

    # 최종 보정 깊이
    corrected_depth = layers.Add()([
        base_depth,
        layers.Multiply()([correction, correction_attention])
    ])

    # 출력 값 범위 제한 (0-1 범위로)
    corrected_depth = layers.Lambda(lambda x: K.clip(x, 0.0, 1.0))(corrected_depth)

    model = Model(inputs, corrected_depth)
    return model

# 단순한 U-Net 모델 생성 함수
def create_simple_unet_model(input_shape=(64, 64, 5)):
    """안정적인 간단한 U-Net 모델"""
    inputs = Input(input_shape)
    
    # Encoder
    conv1 = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    conv1 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = layers.Conv2D(32, (3, 3), padding='same')(conv1)
    conv1 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = layers.Conv2D(64, (3, 3), padding='same')(pool1)
    conv2 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = layers.Conv2D(64, (3, 3), padding='same')(conv2)
    conv2 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # Bottleneck
    conv3 = layers.Conv2D(128, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = layers.Conv2D(128, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv3)
    conv3 = Activation('relu')(conv3)
    
    # Decoder
    up1 = layers.UpSampling2D(size=(2, 2))(conv3)
    up1 = layers.Conv2D(64, (2, 2), padding='same')(up1)
    up1 = BatchNormalization(momentum=0.9, epsilon=1e-5)(up1)
    up1 = Activation('relu')(up1)
    merge1 = layers.Concatenate()([conv2, up1])
    conv4 = layers.Conv2D(64, (3, 3), padding='same')(merge1)
    conv4 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = layers.Conv2D(64, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv4)
    conv4 = Activation('relu')(conv4)
    
    up2 = layers.UpSampling2D(size=(2, 2))(conv4)
    up2 = layers.Conv2D(32, (2, 2), padding='same')(up2)
    up2 = BatchNormalization(momentum=0.9, epsilon=1e-5)(up2)
    up2 = Activation('relu')(up2)
    merge2 = layers.Concatenate()([conv1, up2])
    conv5 = layers.Conv2D(32, (3, 3), padding='same')(merge2)
    conv5 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = layers.Conv2D(32, (3, 3), padding='same')(conv5)
    conv5 = BatchNormalization(momentum=0.9, epsilon=1e-5)(conv5)
    conv5 = Activation('relu')(conv5)
    
    # 출력 - 활성화 함수 변경 (sigmoid -> linear with clip)
    outputs = layers.Conv2D(1, (1, 1), padding='same')(conv5)
    outputs = layers.Lambda(lambda x: tf.clip_by_value(x, 0, 1))(outputs)
    
    model = Model(inputs, outputs)
    return model


# ====================================================================================
# ✅✅✅ 4. 학습 파라미터 및 콜백 조정
# ====================================================================================

# ✅ 학습 중 예측 시각화 콜백 - 중간 결과 확인을 위한 필수 도구
class VisualizePredictions(tf.keras.callbacks.Callback):
    """학습 중 예측 결과를 시각화하는 콜백"""
    def __init__(self, test_data, test_labels, num_samples=3, threshold=0.3):
        self.test_data = test_data[:num_samples]
        self.test_labels = test_labels[:num_samples]
        self.num_samples = num_samples
        self.threshold = threshold
        
    def on_epoch_end(self, epoch, logs=None):
        if epoch % 5 == 0:  # 5 에폭마다 시각화
            predictions = self.model.predict(self.test_data)
            binary_preds = (predictions > self.threshold).astype(np.float32)
            
            fig, axes = plt.subplots(self.num_samples, 3, figsize=(15, 5*self.num_samples))
            for i in range(self.num_samples):
                axes[i, 0].imshow(self.test_data[i].squeeze(), cmap="gray")
                axes[i, 0].set_title("Input")
                axes[i, 1].imshow(self.test_labels[i].squeeze(), cmap="gray")
                axes[i, 1].set_title("Ground Truth")
                axes[i, 2].imshow(binary_preds[i].squeeze(), cmap="gray")
                axes[i, 2].set_title(f"Prediction (Threshold: {self.threshold})")
            
            plt.tight_layout()
            plt.savefig(f'visualizations/pred_epoch_{epoch}.png')
            plt.close()

# ✅ 학습 설정 함수 개선 - 학습률 증가, 콜백 추가
def setup_training(initial_lr=1e-5):  # 학습률도 낮춤
    """
    개선된 학습 설정 - 안정적인 손실 함수 적용
    
    매개변수:
    - initial_lr: 초기 학습률
    """
    # 조기 종료 설정
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=20,
        restore_best_weights=True,
        min_delta=1e-4
    )

    # 학습률 감소 스케줄러
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        verbose=1,
        min_lr=1e-6,
        min_delta=1e-4
    )

    # 체크포인트
    checkpoint = ModelCheckpoint(
        'checkpoints/model.{epoch:02d}-{val_loss:.4f}.h5',
        monitor='val_loss',
        save_best_only=True,
        save_freq='epoch'
    )
    
    # NaN 감지 콜백 추가
    nan_callback = NanLossCallback()
    
    # 차원 호환성 문제를 피하기 위한 컴파일 설정
    optimizer = Adam(
        learning_rate=initial_lr,
        clipnorm=0.5,  # 그래디언트 클리핑 추가
        epsilon=1e-8
    )
    
    return {
        'callbacks': [early_stopping, lr_scheduler, checkpoint, nan_callback],
        'optimizer': optimizer,
        'loss': ultra_stable_depth_loss_v2,  # 안정적인 손실 함수로 변경
        'metrics': ['mae']
    }


def debug_dataset_creation(rgb_dir, depth_array_dir, depth_image_dir, mde_dir, indices):
    print("디버깅: 데이터셋 생성 검사")
    print(f"RGB 디렉토리: {rgb_dir}")
    print(f"깊이 배열 디렉토리: {depth_array_dir}")
    print(f"깊이 이미지 디렉토리: {depth_image_dir}")
    print(f"MDE 디렉토리: {mde_dir}")
    
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_array_files = sorted([f for f in os.listdir(depth_array_dir) if f.endswith('.npy')])
    depth_image_files = sorted([f for f in os.listdir(depth_image_dir) if f.endswith('.png')])
    mde_files = sorted([f for f in os.listdir(mde_dir) if f.endswith('_depth.npy')])
    
    print(f"RGB 파일 수: {len(rgb_files)}")
    print(f"깊이 배열 파일 수: {len(depth_array_files)}")
    print(f"깊이 이미지 파일 수: {len(depth_image_files)}")
    print(f"MDE 파일 수: {len(mde_files)}")
    
    print("\n첫 5개 파일명 예시:")
    print("RGB:", rgb_files[:5])
    print("깊이 배열:", depth_array_files[:5])
    print("깊이 이미지:", depth_image_files[:5])
    print("MDE:", mde_files[:5])

def safe_load_image(path, color_conversion=cv2.COLOR_BGR2RGB):
    """안전한 이미지 로딩"""
    try:
        img = cv2.imread(path)
        if img is None:
            print(f"❌ 이미지 로드 실패: {path}")
            return None
        return cv2.cvtColor(img, color_conversion)
    except Exception as e:
        print(f"❌ 이미지 로드 중 오류: {path}, {e}")
        return None

def safe_load_depth(path, is_npy=False):
    """안전한 깊이 데이터 로딩"""
    try:
        if is_npy:
            depth = np.load(path)
        else:
            depth = cv2.imread(path, cv2.IMREAD_ANYDEPTH)
        
        if depth is None:
            print(f"❌ 깊이 데이터 로드 실패: {path}")
            return None
        return depth
    except Exception as e:
        print(f"❌ 깊이 데이터 로드 중 오류: {path}, {e}")
        return None

def extract_timestamp_safely(filename):
    try:
        # color_20250314_154240.png 형식
        parts = filename.split('_')
        if len(parts) >= 3 and parts[0] == 'color':
            return f"{parts[1]}_{parts[2].replace('.png', '')}"
        return None
    except Exception as e:
        print(f"타임스탬프 추출 오류: {filename}, {e}")
        return None

def robust_dataset_loading(rgb_dir, depth_array_dir, depth_image_dir, mde_dir, indices, target_size=(64, 64)):
    """개선된 데이터셋 로딩 함수"""
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.startswith('color_') and f.endswith('.png')])
    
    rgb_images = []
    depth_maps = []
    mde_maps = []
    
    for index in indices:
        if index >= len(rgb_files):
            print(f"❌ 인덱스 {index}는 파일 범위를 벗어났습니다.")
            continue
        
        rgb_filename = rgb_files[index]
        timestamp = extract_timestamp_safely(rgb_filename)
        
        if not timestamp:
            continue
        
        # RGB 이미지 로드
        rgb_path = os.path.join(rgb_dir, rgb_filename)
        rgb = safe_load_image(rgb_path)
        if rgb is None:
            continue
        
        # 깊이 맵 로드 (우선순위: NPY > PNG)
        depth_npy_path = os.path.join(depth_array_dir, f'depth_{timestamp}.npy')
        depth_png_path = os.path.join(depth_image_dir, f'depth_{timestamp}.png')
        
        depth = safe_load_depth(depth_npy_path, is_npy=True)
        if depth is None:
            depth = safe_load_depth(depth_png_path)
        
        if depth is None:
            continue
        
        # MDE 맵 로드
        mde_path = os.path.join(mde_dir, f'color_{timestamp}_depth.npy')
        mde = safe_load_depth(mde_path, is_npy=True)
        
        if mde is None:
            continue
        
        # 크기 조정
        rgb_resized = cv2.resize(rgb, target_size, interpolation=cv2.INTER_AREA)
        depth_resized = cv2.resize(depth, target_size, interpolation=cv2.INTER_NEAREST)
        mde_resized = cv2.resize(mde, target_size, interpolation=cv2.INTER_NEAREST)
        
        rgb_images.append(rgb_resized)
        depth_maps.append(depth_resized)
        mde_maps.append(mde_resized)
    
    if not rgb_images:
        print("❌ 유효한 샘플이 없습니다!")
        return None
    
    return (
        np.array(rgb_images), 
        np.array(depth_maps), 
        np.array(mde_maps)
    )

def enhanced_dataset_creator(
    rgb_dir, 
    depth_array_dir, 
    depth_image_dir, 
    mde_dir, 
    indices, 
    batch_size=8, 
    target_size=(64, 64)
):
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.startswith('color_') and f.endswith('.png')])
    
    rgb_images, depth_maps, mde_maps = [], [], []
    
    for idx in indices:
        if idx >= len(rgb_files):
            continue
        
        rgb_filename = rgb_files[idx]
        timestamp = extract_timestamp_safely(rgb_filename)
        
        if not timestamp:
            continue
        
        # RGB 이미지 로드
        rgb_path = os.path.join(rgb_dir, rgb_filename)
        rgb = safe_load_image(rgb_path)
        if rgb is None:
            continue
        
        # 깊이 맵 로드
        depth_npy_path = os.path.join(depth_array_dir, f'depth_{timestamp}.npy')
        depth_png_path = os.path.join(depth_image_dir, f'depth_{timestamp}.png')
        
        depth = safe_load_depth(depth_npy_path, is_npy=True)
        if depth is None:
            depth = safe_load_depth(depth_png_path)
        if depth is None:
            continue
        
        # MDE 맵 로드
        mde_path = os.path.join(mde_dir, f'color_{timestamp}_depth.npy')
        mde = safe_load_depth(mde_path, is_npy=True)
        if mde is None:
            continue
        
        # 크기 조정
        rgb_resized = cv2.resize(rgb, target_size, interpolation=cv2.INTER_AREA)
        depth_resized = cv2.resize(depth, target_size, interpolation=cv2.INTER_NEAREST)
        mde_resized = cv2.resize(mde, target_size, interpolation=cv2.INTER_NEAREST)
        
        rgb_images.append(rgb_resized)
        depth_maps.append(depth_resized)
        mde_maps.append(mde_resized)
    
    if not rgb_images:
        print("❌ 유효한 샘플이 없습니다!")
        return None
    
    # 데이터 정규화
    rgb_norm = np.array(rgb_images).astype(np.float32) / 255.0
    depth_norm = np.array(depth_maps).astype(np.float32)
    mde_norm = np.array(mde_maps).astype(np.float32)
    
    # 깊이와 MDE 정규화 (0-1 범위)
    for data in [depth_norm, mde_norm]:
        valid_mask = data > 0
        if np.sum(valid_mask) > 0:
            min_val = np.min(data[valid_mask])
            max_val = np.max(data[valid_mask])
            if max_val > min_val:
                data[valid_mask] = (data[valid_mask] - min_val) / (max_val - min_val)
    
    # 5채널 입력 생성
    inputs = np.concatenate([
        rgb_norm,
        np.expand_dims(depth_norm, axis=-1),
        np.expand_dims(mde_norm, axis=-1)
    ], axis=-1)
    
    targets = np.expand_dims(depth_norm, axis=-1)
    
    print(f"입력 데이터 형태: {inputs.shape}")
    print(f"타겟 데이터 형태: {targets.shape}")
    
    # 데이터셋 생성
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

# 수정된 데이터셋 생성 함수
def create_simple_dataset_with_monitoring(
    rgb_dir, 
    depth_array_dir, 
    depth_image_dir, 
    mde_dir, 
    indices, 
    batch_size, 
    target_size=(64, 64),
    stage=0  # Stage 인자를 추가하여 입력 채널 수를 조정
):
    """
    데이터셋 생성 및 모니터링 함수
    stage: 학습 단계 (0이면 5채널, 1 이상이면 7채널)
    """
    print(f"데이터셋 생성 중... 대상 해상도: {target_size}, Stage: {stage}")
    
    rgb_images = []
    depth_maps = []
    mde_predictions = []

    # 데이터 로드
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    for i in indices:
        if i >= len(rgb_files):
            print(f"❌ 인덱스 {i}는 RGB 파일 범위를 벗어났습니다.")
            continue
        
        rgb_filename = rgb_files[i]
        timestamp = rgb_filename.replace('color_', '').replace('.png', '')
        
        # RGB 이미지 로드
        rgb_path = os.path.join(rgb_dir, rgb_filename)
        try:
            rgb = cv2.imread(rgb_path)
            if rgb is None:
                print(f"❌ RGB 이미지 로드 실패: {rgb_path}")
                continue
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"❌ RGB 이미지 로드 중 오류: {e}")
            continue
        
        # 깊이 맵 로드
        depth_array_path = os.path.join(depth_array_dir, f'depth_{timestamp}.npy')
        depth_image_path = os.path.join(depth_image_dir, f'depth_{timestamp}.png')
        depth = None
        try:
            if os.path.exists(depth_array_path):
                depth = np.load(depth_array_path)
            elif os.path.exists(depth_image_path):
                depth = cv2.imread(depth_image_path, cv2.IMREAD_ANYDEPTH)
            if depth is None:
                print(f"❌ 깊이 데이터 로드 실패: {timestamp}")
                continue
        except Exception as e:
            print(f"❌ 깊이 데이터 로드 중 오류: {e}")
            continue
        
        # MDE 예측 로드
        mde_path = os.path.join(mde_dir, f'color_{timestamp}_depth.npy')
        try:
            if not os.path.exists(mde_path):
                print(f"❌ MDE 파일 없음: {mde_path}")
                continue
            mde = np.load(mde_path)
        except Exception as e:
            print(f"❌ MDE 데이터 로드 중 오류: {e}")
            continue
        
        # 크기 조정
        resize_dims = (target_size[1], target_size[0])  # (너비, 높이)로 변환
        rgb = cv2.resize(rgb, resize_dims, interpolation=cv2.INTER_AREA)
        depth = cv2.resize(depth, resize_dims, interpolation=cv2.INTER_NEAREST)
        mde = cv2.resize(mde, resize_dims, interpolation=cv2.INTER_NEAREST)
        
        rgb_images.append(rgb)
        depth_maps.append(depth)
        mde_predictions.append(mde)
    
    print(f"로드된 샘플 수: {len(rgb_images)}")
    if len(rgb_images) == 0:
        raise ValueError("유효한 샘플이 없습니다!")

    # NumPy 배열로 변환
    rgb_images = np.array(rgb_images)
    depth_maps = np.array(depth_maps)
    mde_predictions = np.array(mde_predictions)

    def preprocess_data(rgb, depth, mde):
        """수치적 안정성이 향상된 데이터 전처리 함수"""
        try:
            # 입력 데이터 유효성 검사
            if (rgb is None or depth is None or mde is None or
                len(rgb.shape) != 3 or len(depth.shape) != 2 or len(mde.shape) != 2):
                print(f"❌ 잘못된 입력 데이터: RGB {rgb.shape if rgb is not None else None}, "
                      f"깊이 {depth.shape if depth is not None else None}, "
                      f"MDE {mde.shape if mde is not None else None}")
                return None, None
    
            # 전처리 함수 추가
            def enhanced_preprocessing(data):
                data = np.nan_to_num(data, nan=0.0, posinf=0.0, neginf=0.0)
                data = np.clip(data, 0.0, 1.0)
                data[data < 1e-7] = 0.0
                return data
    
            # RGB 정규화
            rgb_norm = rgb.astype(np.float32) / 255.0
            rgb_norm = enhanced_preprocessing(rgb_norm)
    
            # 깊이 맵 정규화
            valid_depth_mask = depth > 0
            depth_normalized = np.zeros_like(depth, dtype=np.float32)
            if np.sum(valid_depth_mask) > 0:
                depth_min = np.min(depth[valid_depth_mask])
                depth_max = np.max(depth[valid_depth_mask])
                if depth_max > depth_min:
                    depth_normalized[valid_depth_mask] = (depth[valid_depth_mask] - depth_min) / (depth_max - depth_min)
            depth_normalized = enhanced_preprocessing(depth_normalized)
    
            # MDE 맵 정규화
            mde_valid_mask = mde > 0
            mde_normalized = np.zeros_like(mde, dtype=np.float32)
            if np.sum(mde_valid_mask) > 0:
                mde_min = np.min(mde[mde_valid_mask])
                mde_max = np.max(mde[mde_valid_mask])
                if mde_max > mde_min:
                    mde_normalized[mde_valid_mask] = (mde[mde_valid_mask] - mde_min) / (mde_max - mde_min)
            mde_normalized = enhanced_preprocessing(mde_normalized)
    
            # 신뢰도 맵 생성
            sensor_confidence = np.full_like(depth, 0.7, dtype=np.float32)
            sensor_confidence[~valid_depth_mask] = 0.3  # 깊이 값이 없는 픽셀에 낮은 신뢰도
            mde_confidence = np.full_like(mde, 0.7, dtype=np.float32)
            mde_confidence[~mde_valid_mask] = 0.3      # MDE 값이 없는 픽셀에 낮은 신뢰도
    
            # 입력 결합
            if stage == 0:  # Stage 1: 5채널
                combined_input = np.concatenate([
                    rgb_norm,
                    np.expand_dims(depth_normalized, axis=-1),
                    np.expand_dims(mde_normalized, axis=-1)
                ], axis=-1)
            else:  # Stage 2 이상: 7채널
                combined_input = np.concatenate([
                    rgb_norm,
                    np.expand_dims(depth_normalized, axis=-1),
                    np.expand_dims(mde_normalized, axis=-1),
                    np.expand_dims(sensor_confidence, axis=-1),
                    np.expand_dims(mde_confidence, axis=-1)
                ], axis=-1)
        
            # 타겟 깊이
            target_depth = np.expand_dims(depth_normalized, axis=-1)
            return combined_input, target_depth
    
        except Exception as e:
            print(f"데이터 전처리 중 오류 발생: {e}")
            return None, None
    
    print("데이터 전처리 시작")
    inputs = []
    targets = []
    
    for i in range(len(rgb_images)):
        x, y = preprocess_data(rgb_images[i], depth_maps[i], mde_predictions[i])
        if x is not None and y is not None:
            inputs.append(x)
            targets.append(y)
    
    if len(inputs) == 0:
        print("❌ 전처리 후 유효한 샘플이 없습니다!")
        return None
    
    # NumPy 배열로 변환
    inputs = np.array(inputs)
    targets = np.array(targets)

    # 입력 데이터 최종 확인
    print("최종 입력 데이터 형태:", inputs.shape)
    print("입력 데이터 범위:")
    print("  RGB:", np.min(inputs[:,:,:,:3]), "~", np.max(inputs[:,:,:,:3]))
    print("  깊이:", np.min(inputs[:,:,:,3]), "~", np.max(inputs[:,:,:,3]))
    print("  MDE:", np.min(inputs[:,:,:,4]), "~", np.max(inputs[:,:,:,4]))
    if stage > 0:
        print("  Sensor Confidence:", np.min(inputs[:,:,:,5]), "~", np.max(inputs[:,:,:,5]))
        print("  MDE Confidence:", np.min(inputs[:,:,:,6]), "~", np.max(inputs[:,:,:,6]))
    print("최종 타겟 데이터 형태:", targets.shape)
    print("타겟 깊이 범위:", np.min(targets), "~", np.max(targets))

    # TensorFlow 데이터셋 생성
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

    return dataset


def plot_evaluation_metrics(metrics):
    """
    깊이 예측 평가 메트릭을 시각화하는 함수
    
    매개변수:
    - metrics: 평가 메트릭 딕셔너리
    """
    plt.figure(figsize=(15, 5))
    
    # 각 메트릭에 대한 박스 플롯과 히스토그램
    metric_names = list(metrics.keys())
    
    # 박스 플롯
    plt.subplot(121)
    plt.boxplot([metrics[name] for name in metric_names], labels=metric_names)
    plt.title('Depth Prediction Metrics - Box Plot')
    plt.ylabel('Metric Value')
    plt.xticks(rotation=45)
    
    # 히스토그램
    plt.subplot(122)
    for name in metric_names:
        plt.hist(metrics[name], alpha=0.5, label=name, bins=10)
    plt.title('Depth Prediction Metrics - Histogram')
    plt.xlabel('Metric Value')
    plt.ylabel('Frequency')
    plt.legend(loc='best')
    
    plt.tight_layout()
    plt.savefig('visualizations/depth_metrics_distribution.png')
    plt.close()

def visualize_results(rgb, true_depth, hybrid_depth, confidence, pred_depth, error):
    """
    깊이 예측 결과를 시각화하는 함수
    
    매개변수:
    - rgb: RGB 이미지
    - true_depth: 실제 깊이 맵
    - hybrid_depth: 하이브리드 깊이 맵 (입력)
    - confidence: 신뢰도 맵
    - pred_depth: 예측된 깊이 맵
    - error: 깊이 오차 맵
    """
    plt.figure(figsize=(15, 10))
    
    # RGB 이미지
    plt.subplot(231)
    plt.imshow(rgb.astype(np.uint8))
    plt.title('RGB Image')
    plt.axis('off')
    
    # 실제 깊이 맵
    plt.subplot(232)
    plt.imshow(true_depth, cmap='viridis')
    plt.title('Ground Truth Depth')
    plt.colorbar(label='Depth')
    plt.axis('off')
    
    # 하이브리드 깊이 맵 (입력)
    plt.subplot(233)
    plt.imshow(hybrid_depth, cmap='viridis')
    plt.title('Hybrid Depth (Input)')
    plt.colorbar(label='Depth')
    plt.axis('off')
    
    # 신뢰도 맵
    plt.subplot(234)
    plt.imshow(confidence, cmap='gray')
    plt.title('Confidence Map')
    plt.colorbar(label='Confidence')
    plt.axis('off')
    
    # 예측된 깊이 맵
    plt.subplot(235)
    plt.imshow(pred_depth, cmap='viridis')
    plt.title('Predicted Depth')
    plt.colorbar(label='Depth')
    plt.axis('off')
    
    # 깊이 오차 맵
    plt.subplot(236)
    plt.imshow(error, cmap='hot')
    plt.title('Depth Error')
    plt.colorbar(label='Error')
    plt.axis('off')
    
    plt.tight_layout()
    
    return plt

def evaluate_and_visualize_depth_results(model, test_dataset, num_samples=5):
    """테스트 데이터셋에 대한 깊이 보정 결과 평가 및 시각화"""
    # 메트릭 초기화
    metrics = {
        'RMSE': [],
        'MAE': [],
        '상대 오차': [],
        '델타(δ) < 1.25': [],
        '델타(δ) < 1.25²': [],
        '델타(δ) < 1.25³': []
    }
    
    # 디버깅: 데이터셋 확인
    print("테스트 데이터셋 디버깅:")
    for inputs, targets in test_dataset.take(1):
        print("입력 데이터 형태:", inputs.shape)
        print("타겟 데이터 형태:", targets.shape)
    
    # 랜덤 샘플 선택
    random_samples = list(test_dataset.take(num_samples))
    
    for inputs, targets in random_samples:
        try:
            # 모델 예측
            predictions = model.predict(inputs)
            
            print("예측 결과 형태:", predictions.shape)
            
            # 각 샘플에 대해
            for i in range(inputs.shape[0]):
                # 입력 데이터 추출
                rgb = inputs[i, :, :, :3].numpy() * 255.0
                true_depth = targets[i, :, :, 0].numpy()
                
                # 예측 결과 안전하게 추출
                if len(predictions.shape) == 4:
                    pred_depth = predictions[i, :, :, 0]
                elif len(predictions.shape) == 3:
                    pred_depth = predictions[i, :, :]
                else:
                    print(f"예측 결과 형태 오류: {predictions.shape}")
                    continue
                
                # 유효한 깊이 픽셀 마스크
                valid_mask = true_depth > 0
                
                if np.sum(valid_mask) > 0:
                    # 메트릭 계산
                    valid_true_depth = true_depth[valid_mask]
                    valid_pred_depth = pred_depth[valid_mask]
                    
                    # RMSE
                    rmse = np.sqrt(np.mean((valid_true_depth - valid_pred_depth)**2))
                    metrics['RMSE'].append(rmse)
                    
                    # MAE
                    mae = np.mean(np.abs(valid_true_depth - valid_pred_depth))
                    metrics['MAE'].append(mae)
                    
                    # 상대 오차
                    rel_error = np.mean(np.abs(valid_true_depth - valid_pred_depth) / valid_true_depth)
                    metrics['상대 오차'].append(rel_error)
                    
                    # 델타 메트릭
                    ratio = valid_pred_depth / valid_true_depth
                    max_ratio = np.maximum(ratio, 1/ratio)
                    
                    metrics['델타(δ) < 1.25'].append(np.mean(max_ratio < 1.25))
                    metrics['델타(δ) < 1.25²'].append(np.mean(max_ratio < 1.25**2))
                    metrics['델타(δ) < 1.25³'].append(np.mean(max_ratio < 1.25**3))
                    
                    # 시각화
                    plt.figure(figsize=(15, 5))
                    plt.subplot(131)
                    plt.title('RGB')
                    plt.imshow(rgb.astype(np.uint8))
                    plt.axis('off')
                    
                    plt.subplot(132)
                    plt.title('True Depth')
                    plt.imshow(true_depth, cmap='viridis')
                    plt.colorbar()
                    plt.axis('off')
                    
                    plt.subplot(133)
                    plt.title('Predicted Depth')
                    plt.imshow(pred_depth, cmap='viridis')
                    plt.colorbar()
                    plt.axis('off')
                    
                    plt.tight_layout()
                    plt.savefig(f'visualizations/depth_prediction_sample_{i}.png')
                    plt.close()
        
        except Exception as e:
            print(f"샘플 처리 중 오류: {e}")
            import traceback
            traceback.print_exc()
    
    # 메트릭 요약
    print("\n===== 깊이 보정 성능 평가 =====")
    for metric_name, values in metrics.items():
        if values:
            print(f"{metric_name}: {np.mean(values):.4f} ± {np.std(values):.4f}")
    
    return metrics

#데이터셋 크기조정
def resize_dataset(rgb_images, depth_maps, mde_predictions, target_size=(240, 320)):
    """데이터셋의 모든 이미지를 더 작은 해상도로 조정"""
    import cv2
    
    resized_rgb = []
    resized_depth = []
    resized_mde = []
    
    for i in range(len(rgb_images)):
        # RGB 이미지 크기 조정
        rgb = cv2.resize(rgb_images[i], target_size, interpolation=cv2.INTER_AREA)
        
        # 깊이 맵 크기 조정 (INTER_NEAREST를 사용하여 깊이 값 보존)
        depth = cv2.resize(depth_maps[i], target_size, interpolation=cv2.INTER_NEAREST)
        
        # MDE 예측 크기 조정
        mde = cv2.resize(mde_predictions[i], target_size, interpolation=cv2.INTER_NEAREST)
        
        resized_rgb.append(rgb)
        resized_depth.append(depth)
        resized_mde.append(mde)
    
    return np.array(resized_rgb), np.array(resized_depth), np.array(resized_mde)

def setup_mixed_precision(use_mixed_precision=False):
    """혼합 정밀도 학습을 위한 TensorFlow 구성"""
    if use_mixed_precision:
        mixed_precision.set_global_policy('mixed_float16')
        print('혼합 정밀도 정책 설정: mixed_float16')
    else:
        mixed_precision.set_global_policy('float32')
        print('기본 정밀도 정책 설정: float32')

def plot_training_history(histories, stages):
    """
    다중 해상도 학습 단계의 훈련 히스토리를 시각화하는 함수
    
    매개변수:
    - histories: 각 학습 단계의 히스토리 리스트
    - stages: 각 학습 단계의 해상도 리스트
    """
    plt.figure(figsize=(15, 10))
    
    # 손실(Loss) 그래프
    plt.subplot(2, 1, 1)
    for i, (history, stage) in enumerate(zip(histories, stages)):
        plt.plot(history.history['loss'], label=f'Train Loss (Stage {i+1}: {stage})', alpha=0.7)
        plt.plot(history.history['val_loss'], label=f'Validation Loss (Stage {i+1}: {stage})', linestyle='--', alpha=0.7)
    
    plt.title('Training and Validation Loss across Stages')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='best')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # MAE(Mean Absolute Error) 그래프
    plt.subplot(2, 1, 2)
    for i, (history, stage) in enumerate(zip(histories, stages)):
        plt.plot(history.history['mae'], label=f'Train MAE (Stage {i+1}: {stage})', alpha=0.7)
        plt.plot(history.history['val_mae'], label=f'Validation MAE (Stage {i+1}: {stage})', linestyle='--', alpha=0.7)
    
    plt.title('Training and Validation Mean Absolute Error across Stages')
    plt.xlabel('Epoch')
    plt.ylabel('Mean Absolute Error')
    plt.legend(loc='best')
    plt.grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('visualizations/training_history.png')
    plt.close()

def transfer_weights_safely_improved(old_model, new_model):
    """개선된 안전한 가중치 전이 함수 - skip_mismatch 옵션 사용"""
    print("개선된 가중치 전이 함수 실행 중...")
    
    # 임시 가중치 파일 저장
    temp_weights_path = 'temp_weights.h5'
    old_model.save_weights(temp_weights_path)
    
    try:
        # by_name=True와 skip_mismatch=True 옵션으로 가중치 로드
        new_model.load_weights(
            temp_weights_path, 
            by_name=True, 
            skip_mismatch=True
        )
        print("가중치 전이 성공! (일치하지 않는 레이어는 건너뜀)")
        return True
    except Exception as e:
        print(f"가중치 전이 오류: {e}")
        return False
    finally:
        # 임시 파일 삭제 시도
        try:
            if os.path.exists(temp_weights_path):
                os.remove(temp_weights_path)
        except:
            pass

# 깊이 보정 결과 시각화 콜백 클래스 (전역 범위로 이동)
class DepthCorrectionVisualizer(tf.keras.callbacks.Callback):
    def __init__(self, dataset, log_dir, sample_count=2, save_freq=2):
        super().__init__()
        self.dataset = dataset
        self.log_dir = log_dir
        self.sample_count = sample_count
        self.save_freq = save_freq
        self.sample_inputs = []
        self.sample_targets = []
        for batch in self.dataset.take(sample_count):
            inputs, targets = batch
            self.sample_inputs.append(inputs[:1])
            self.sample_targets.append(targets[:1])
        os.makedirs(log_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.save_freq != 0 or not self.sample_inputs:
            return

        plt.figure(figsize=(12, 4 * self.sample_count))
        for i, (sample_input, sample_target) in enumerate(zip(self.sample_inputs, self.sample_targets)):
            try:
                sample_pred = self.model.predict(sample_input)
                rgb = sample_input[0, :, :, :3].numpy()  # RGB만 추출
                true_depth = sample_target[0, :, :, 0].numpy()
                pred_depth = sample_pred[0, :, :, 0]

                plt.subplot(self.sample_count, 3, 3*i + 1)
                plt.imshow(rgb)
                plt.title('RGB')
                plt.axis('off')

                plt.subplot(self.sample_count, 3, 3*i + 2)
                plt.imshow(true_depth, cmap='viridis')
                plt.title('Ground Truth')
                plt.axis('off')

                plt.subplot(self.sample_count, 3, 3*i + 3)
                plt.imshow(pred_depth, cmap='viridis')
                plt.title('Prediction')
                plt.axis('off')
            except Exception as e:
                print(f"Error processing sample {i}: {e}")
        plt.tight_layout()
        plt.savefig(f'{self.log_dir}/vis_epoch_{epoch:03d}.png', dpi=100)
        plt.close()

# 개선된 학습 콜백 함수
def get_improved_callbacks():
    """개선된 학습 콜백 함수"""
    # 더 긴 인내심으로 조기 종료 설정
    early_stopping = EarlyStopping(
        monitor='val_loss',
        patience=15,  # 늘림
        restore_best_weights=True,
        min_delta=0.0005  # 더 작은 변화에도 반응
    )

    # 더 점진적인 학습률 감소
    lr_scheduler = ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.8,  # 더 작은 감소 계수
        patience=10,  # 늘림
        verbose=1,
        min_lr=1e-7  # 더 낮은 최소 학습률
    )

    # 체크포인트
    checkpoint = ModelCheckpoint(
        'checkpoints/depth_model_{epoch:02d}_{val_loss:.6f}.h5',
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False  # 전체 모델 저장
    )
    
    # 텐서보드 로깅 추가
    tensorboard = tf.keras.callbacks.TensorBoard(
        log_dir='./logs',
        histogram_freq=1,
        write_graph=True,
        update_freq='epoch'
    )
    
    return [early_stopping, lr_scheduler, checkpoint, tensorboard]

# 테스트 결과 저장 함수
def save_test_results(test_results, metrics):
    """테스트 결과를 파일로 저장"""
    results_dir = 'test_results'
    os.makedirs(results_dir, exist_ok=True)
    
    with open(f'{results_dir}/test_metrics.txt', 'w') as f:
        f.write("===== 깊이 보정 모델 테스트 결과 =====\n\n")
        f.write(f"테스트 손실: {test_results[0]:.4f}\n")
        f.write(f"테스트 MAE: {test_results[1]:.4f}\n\n")
        
        f.write("개별 메트릭 평균:\n")
        for metric_name, values in metrics.items():
            if values:
                mean_val = np.mean(values)
                std_val = np.std(values)
                f.write(f"{metric_name}: {mean_val:.4f} ± {std_val:.4f}\n")
    
    print(f"✅ 테스트 결과가 저장되었습니다: {results_dir}/test_metrics.txt")

# 모델 컴파일 함수 수정
def compile_model_with_gradient_clipping(model, learning_rate=1e-5):
    """안정적인 학습을 위한 모델 컴파일"""
    # 개선된 옵티마이저 사용
    optimizer = SafeAdamOptimizer(
        learning_rate=learning_rate,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-7
    )

    model.compile(
        optimizer=optimizer,
        loss=ultra_stable_depth_loss_v2,  # 새로운 손실 함수
        metrics=['mae']
    )

    return model

# 모델 학습 직전에 NaN 탐지 함수 추가
def detect_nan_values(dataset, name="데이터셋"):
    for batch_idx, (inputs, targets) in enumerate(dataset):
        inputs_np = inputs.numpy()
        targets_np = targets.numpy()
        
        if np.isnan(inputs_np).any() or np.isinf(inputs_np).any():
            print(f"{name} 입력 배치 {batch_idx}에 NaN/Inf 값이 있습니다!")
            return False
        if np.isnan(targets_np).any() or np.isinf(targets_np).any():
            print(f"{name} 타겟 배치 {batch_idx}에 NaN/Inf 값이 있습니다!")
            return False
    print(f"{name}에 NaN/Inf 값이 감지되지 않았습니다.")
    return True

def plot_training_metrics(metrics_history):
    """학습 과정의 메트릭을 시각화"""
    plt.figure(figsize=(15, 12))
    
    # 1. 훈련 및 검증 손실
    plt.subplot(2, 2, 1)
    plt.plot(metrics_history['train_loss'], label='Train Loss')
    plt.plot(metrics_history['val_loss'], label='Validation Loss')
    
    # 해상도 전환 구간 표시
    res_changes = []
    prev_res = None
    for i, res in enumerate(metrics_history['resolutions']):
        if res != prev_res:
            res_changes.append(i)
            prev_res = res
    
    for idx in res_changes[1:]:
        plt.axvline(x=idx, color='r', linestyle='--', alpha=0.3)
    
    plt.title('Loss over Training')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 2. 훈련 및 검증 MAE
    plt.subplot(2, 2, 2)
    plt.plot(metrics_history['train_mae'], label='Train MAE')
    plt.plot(metrics_history['val_mae'], label='Validation MAE')
    
    for idx in res_changes[1:]:
        plt.axvline(x=idx, color='r', linestyle='--', alpha=0.3)
    
    plt.title('Mean Absolute Error over Training')
    plt.xlabel('Epoch')
    plt.ylabel('MAE')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 3. 학습률 변화
    plt.subplot(2, 2, 3)
    plt.plot(metrics_history['learning_rates'])
    
    for idx in res_changes[1:]:
        plt.axvline(x=idx, color='r', linestyle='--', alpha=0.3)
    
    plt.title('Learning Rate over Training')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.grid(True, alpha=0.3)
    
    # 4. 해상도 변화 표시
    plt.subplot(2, 2, 4)
    # 해상도별 구간 막대 그래프
    unique_res = []
    res_counts = []
    current_res = None
    count = 0
    
    for res in metrics_history['resolutions']:
        if res != current_res:
            if current_res is not None:
                unique_res.append(current_res)
                res_counts.append(count)
            current_res = res
            count = 1
        else:
            count += 1
    
    # 마지막 해상도 추가
    if current_res is not None:
        unique_res.append(current_res)
        res_counts.append(count)
    
    plt.bar(unique_res, res_counts)
    plt.title('Epochs per Resolution')
    plt.xlabel('Resolution')
    plt.ylabel('Epochs')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('visualizations/training_metrics.png', dpi=150)
    plt.close()
    print("✅ 학습 메트릭 시각화가 저장되었습니다: visualizations/training_metrics.png")

# NaN 감지 콜백 (이미 있는 코드)
class NanLossCallback(tf.keras.callbacks.Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is None:
            logs = {}
        loss = logs.get('loss')
        if loss is not None and (np.isnan(loss) or np.isinf(loss)):
            print(f"NaN/Inf 손실 감지됨: {loss}, 학습 중지")
            self.model.stop_training = True
            
    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        loss = logs.get('loss')
        if loss is not None and (np.isnan(loss) or np.isinf(loss)):
            print(f"에폭 {epoch}에서 NaN/Inf 손실 감지됨: {loss}, 학습 중지")
            self.model.stop_training = True

def debug_dataset_loading(rgb_dir, depth_array_dir, depth_image_dir, mde_dir, indices, target_size=(64, 64)):
    """
    데이터셋 로딩 과정을 상세히 디버깅하는 함수
    """
    print("데이터셋 로딩 디버깅 시작")
    
    # 파일 목록
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    depth_array_files = sorted([f for f in os.listdir(depth_array_dir) if f.endswith('.npy')])
    depth_image_files = sorted([f for f in os.listdir(depth_image_dir) if f.endswith('.png')])
    mde_files = sorted([f for f in os.listdir(mde_dir) if f.endswith('_depth.npy')])
    
    print(f"RGB 파일 수: {len(rgb_files)}")
    print(f"깊이 배열 파일 수: {len(depth_array_files)}")
    print(f"깊이 이미지 파일 수: {len(depth_image_files)}")
    print(f"MDE 파일 수: {len(mde_files)}")
    
    # 디버깅용 로깅
    detailed_errors = []
    
    loaded_samples = 0
    for i in indices:
        if i >= len(rgb_files) or i >= len(mde_files):
            detailed_errors.append(f"인덱스 {i}는 파일 범위를 벗어났습니다.")
            continue
        
        try:
            # RGB 이미지 로드
            rgb_path = os.path.join(rgb_dir, rgb_files[i])
            rgb = cv2.imread(rgb_path)
            if rgb is None:
                detailed_errors.append(f"RGB 이미지 로드 실패: {rgb_path}")
                continue
            rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
            
            # 파일명에서 타임스탬프 추출
            timestamp = None
            parts = rgb_files[i].split('_')
            if len(parts) >= 3 and parts[0] == 'color':
                timestamp = f"{parts[1]}_{parts[2].replace('.png', '')}"
            else:
                detailed_errors.append(f"RGB 파일명 형식 오류: {rgb_files[i]}")
                continue
            
            # 깊이 맵 로드
            depth_array_path = os.path.join(depth_array_dir, f"depth_{timestamp}.npy")
            depth_image_path = os.path.join(depth_image_dir, f"depth_{timestamp}.png")
            
            depth = None
            if os.path.exists(depth_array_path):
                depth = np.load(depth_array_path)
            elif os.path.exists(depth_image_path):
                depth = cv2.imread(depth_image_path, cv2.IMREAD_ANYDEPTH)
            
            if depth is None:
                detailed_errors.append(f"타임스탬프 {timestamp}에 대한 깊이 데이터 없음")
                continue
            
            # MDE 예측 로드
            mde_path = os.path.join(mde_dir, f"color_{timestamp}_depth.npy")
            if not os.path.exists(mde_path):
                detailed_errors.append(f"MDE 파일 없음: {mde_path}")
                continue
            
            mde = np.load(mde_path)
            
            # 크기 조정
            rgb_resized = cv2.resize(rgb, target_size, interpolation=cv2.INTER_AREA)
            depth_resized = cv2.resize(depth, target_size, interpolation=cv2.INTER_NEAREST)
            mde_resized = cv2.resize(mde, target_size, interpolation=cv2.INTER_NEAREST)
            
            # 데이터 형태 및 범위 확인
            print(f"샘플 {loaded_samples} 데이터 형태:")
            print(f"  RGB: {rgb_resized.shape}, 타입: {rgb_resized.dtype}")
            print(f"  깊이: {depth_resized.shape}, 타입: {depth_resized.dtype}")
            print(f"  MDE: {mde_resized.shape}, 타입: {mde_resized.dtype}")
            print(f"  깊이 범위: {np.min(depth_resized)} ~ {np.max(depth_resized)}")
            print(f"  MDE 범위: {np.min(mde_resized)} ~ {np.max(mde_resized)}")
            
            loaded_samples += 1
            
        except Exception as e:
            detailed_errors.append(f"샘플 {i} 처리 중 오류: {str(e)}")
    
    print(f"총 로드된 샘플 수: {loaded_samples}")
    
    if detailed_errors:
        print("\n상세 오류:")
        for error in detailed_errors:
            print(error)
    
    return loaded_samples > 0

def progressive_resolution_training_improved(
    rgb_dir, depth_array_dir, depth_image_dir, mde_dir, train_idx, val_idx, test_idx
):
    """
    개선된 점진적 해상도 훈련 함수
    - 안정성 강화
    - 학습 진행 상황 시각화 개선
    - 결과 분석 기능 추가
    """
    # 낮은 해상도에서 시작하여 점진적으로 증가
    resolutions = [(32, 32), (48, 64), (96, 128), (240, 320), (480, 640)]
    epochs_per_stage = [15, 12, 10, 8, 5]  # 점진적으로 줄임
    batch_sizes = [16, 12, 8, 4, 2]  # 640x480에서는 배치 크기를 2로 줄임
    
    # 안정적인 학습을 위한 학습률 감소
    base_lr = 1e-5
    lr_factors = [1.0, 0.1, 0.05, 0.02]
    
    # 학습/검증/테스트 분할 확인
    print(f"학습 샘플: {len(train_idx)}, 검증 샘플: {len(val_idx)}, 테스트 샘플: {len(test_idx)}")
    
    # 로그 디렉토리 생성
    log_dir = "training_logs"
    os.makedirs(log_dir, exist_ok=True)
    
    # 모니터링 메트릭 저장
    metrics_history = {
        'train_loss': [],
        'val_loss': [],
        'train_mae': [],
        'val_mae': [],
        'learning_rates': [],
        'resolutions': []
    }
    
    # 모델 초기화
    model = None
    
    for stage, (resolution, epochs, batch_size, lr_factor) in enumerate(zip(resolutions, epochs_per_stage, batch_sizes, lr_factors)):
        stage_name = f"Stage{stage+1}_{resolution[0]}x{resolution[1]}"
        print(f"\n{'='*20} 학습 단계 {stage+1}/{len(resolutions)}: 해상도 {resolution} {'='*20}")
        
        # 현재 단계의 학습률 계산
        current_lr = base_lr * lr_factor
        print(f"현재 학습률: {current_lr:.2e}")
    
        try:
            # 데이터셋 생성
            print(f"데이터셋 생성 중... 대상 해상도: {resolution}")
            train_ds = create_simple_dataset_with_monitoring(
                rgb_dir, depth_array_dir, depth_image_dir, mde_dir, train_idx, batch_size, target_size=resolution, stage=stage
            )
            val_ds = create_simple_dataset_with_monitoring(
                rgb_dir, depth_array_dir, depth_image_dir, mde_dir, val_idx, batch_size, target_size=resolution, stage=stage
            )

            # 데이터셋 생성 실패 시 종료
            if train_ds is None or val_ds is None:
                raise ValueError(f"Stage {stage+1}에서 데이터셋 생성에 실패했습니다.")

            # 데이터셋 배치 형태 확인
            for batch in train_ds.take(1):
                inputs, targets = batch
                print(f"학습 데이터 배치 형태: 입력={inputs.shape}, 타겟={targets.shape}")
            
            # 데이터셋 NaN 검사
            print("데이터셋 NaN/Inf 검사 중...")
            train_data_valid = detect_nan_values(train_ds, "학습 데이터셋")
            val_data_valid = detect_nan_values(val_ds, "검증 데이터셋")
            if not (train_data_valid and val_data_valid):
                raise ValueError(f"Stage {stage+1}에서 데이터셋에 NaN/Inf 값이 발견되었습니다.")
        
            # 모델 생성 또는 조정
            input_shape = (resolution[0], resolution[1], 5 if stage == 0 else 7)
            print(f"모델 입력 차원: {input_shape}")
            
            if model is None or stage == 0:
                if stage == 0:
                    model = create_simple_unet_model(input_shape)
                else:
                    model = create_mde_aware_correction_model_with_attention(input_shape)
                model = compile_model_with_gradient_clipping(model, learning_rate=current_lr)
                print("새 모델 생성 완료")
            else:
                old_model_weights_path = f'checkpoints/depth_model_stage{stage}_weights.h5'
                model.save_weights(old_model_weights_path)
                new_model = create_mde_aware_correction_model_with_attention(input_shape)
                new_model = compile_model_with_gradient_clipping(new_model, learning_rate=current_lr)
                transfer_success = transfer_weights_safely_improved(model, new_model)
                if transfer_success:
                    print("가중치 전이 성공")
                else:
                    print("가중치 전이 실패, 새 가중치로 시작합니다")
                model = new_model
            
            # 모델 요약 출력 (디버깅용)
            model.summary(print_fn=lambda x: print(x))
            
            # 체크포인트 디렉토리
            stage_checkpoint_dir = f'checkpoints/stage{stage+1}'
            os.makedirs(stage_checkpoint_dir, exist_ok=True)
            
            # 콜백 설정
            callbacks = [
                tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss',
                    patience=10,
                    restore_best_weights=True,
                    min_delta=0.0005
                ),
                tf.keras.callbacks.ReduceLROnPlateau(
                    monitor='val_loss',
                    factor=0.7,
                    patience=5,
                    verbose=1,
                    min_lr=1e-7
                ),
                tf.keras.callbacks.ModelCheckpoint(
                    f'{stage_checkpoint_dir}/model_epoch_{{epoch:02d}}_valloss_{{val_loss:.4f}}.h5',
                    monitor='val_loss',
                    save_best_only=True
                ),
                NanLossCallback(),
                tf.keras.callbacks.TensorBoard(
                    log_dir=f'{log_dir}/{stage_name}',
                    histogram_freq=1,
                    update_freq='epoch'
                ),
                DepthCorrectionVisualizer(
                    dataset=val_ds,
                    log_dir=f'visualizations/stage{stage+1}',
                    sample_count=2,
                    save_freq=2
                )
            ]
            
            # 학습 시작
            print(f"학습 시작: 단계 {stage+1}, 해상도 {resolution}, 에폭 {epochs}")
            history = model.fit(
                train_ds,
                epochs=epochs,
                validation_data=val_ds,
                callbacks=callbacks,
                verbose=1
            )
            
            # 메트릭 저장
            metrics_history['train_loss'].extend(history.history['loss'])
            metrics_history['val_loss'].extend(history.history['val_loss'])
            metrics_history['train_mae'].extend(history.history['mae'])
            metrics_history['val_mae'].extend(history.history['val_mae'])
            metrics_history['learning_rates'].extend([current_lr] * len(history.history['loss']))
            metrics_history['resolutions'].extend([f"{resolution[0]}x{resolution[1]}"] * len(history.history['loss']))
            
            # 학습 결과 출력
            print(f"단계 {stage+1} 최종 결과:")
            print(f"  - 훈련 손실: {history.history['loss'][-1]:.4f}")
            print(f"  - 검증 손실: {history.history['val_loss'][-1]:.4f}")
            print(f"  - 훈련 MAE: {history.history['mae'][-1]:.4f}")
            print(f"  - 검증 MAE: {history.history['val_mae'][-1]:.4f}")
            
            # 단계별 모델 저장
            stage_model_path = f'checkpoints/depth_model_stage{stage+1}.h5'
            model.save(stage_model_path)
            print(f"단계 {stage+1} 모델 저장 완료: {stage_model_path}")
            
        except Exception as e:
            print(f"⚠️ 학습 단계 {stage+1} 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            raise  # 오류 발생 시 종료
    
    # 학습 진행 상황 시각화
    plot_training_metrics(metrics_history)
    
    # 최종 모델 저장
    try:
        final_model_path = 'depth_correction_model_final.h5'
        model.save(final_model_path)
        print(f"✅ 최종 모델이 저장되었습니다: {final_model_path}")
    except Exception as e:
        print(f"❌ 최종 모델 저장 중 오류 발생: {e}")
    
    # 테스트 세트로 최종 평가
    print("\n===== 최종 모델 평가 =====")
    test_ds = create_simple_dataset_with_monitoring(
        rgb_dir, depth_array_dir, depth_image_dir, mde_dir, test_idx, batch_sizes[-1], target_size=resolutions[-1], stage=len(resolutions)-1
    )
    
    try:
        test_results = model.evaluate(test_ds, verbose=1)
        print(f"테스트 손실: {test_results[0]:.4f}, MAE: {test_results[1]:.4f}")
        metrics = evaluate_and_visualize_depth_results(model, test_ds, num_samples=5)
        save_test_results(test_results, metrics)
    except Exception as e:
        print(f"❌ 테스트 평가 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()
    
    return model, train_idx, val_idx, test_idx

# 실행 함수
def run_improved_depth_correction_pipeline():
    """개선된 깊이 보정 파이프라인 실행 함수"""
    print("\n" + "="*80)
    print("🚀 개선된 깊이 보정 파이프라인 시작")
    print("="*80)
    
    # 시작 시간 기록
    start_time = time.time()
    
    # 시각화 및 체크포인트 디렉토리 생성
    for directory in ['visualizations', 'checkpoints', 'logs', 'test_results']:
        os.makedirs(directory, exist_ok=True)
        print(f"📁 {directory} 디렉토리 확인 완료")
    
    # 파일 경로 설정
    rgb_dir = 'color_depth_images/color'
    depth_image_dir = 'color_depth_images/depth'  # 깊이 이미지 디렉토리
    depth_array_dir = 'color_depth_images/depth_npz'  # 깊이 배열 디렉토리
    mde_dir = 'color_depth_images/mde'
    
    # 사용 가능한 샘플 확인
    rgb_files = sorted([f for f in os.listdir(rgb_dir) if f.endswith('.png')])
    print(f"📊 발견된 RGB 이미지: {len(rgb_files)}개")
    
    # 샘플 개수 설정 (전체 파일 수와 지정 값 중 작은 값)
    num_samples = min(1000, len(rgb_files))
    print(f"📊 사용할 샘플 수: {num_samples}개")
    
    # 혼합 정밀도 설정 (float32로 고정)
    print("🔧 정밀도 설정: float32 (안정성 향상)")
    mixed_precision.set_global_policy('float32')
    
    # 학습/검증/테스트 분할
    from sklearn.model_selection import train_test_split
    indices = np.arange(num_samples)
    train_idx, temp_idx = train_test_split(indices, test_size=0.4, random_state=42)
    val_idx, test_idx = train_test_split(temp_idx, test_size=0.5, random_state=42)
    
    # 점진적 해상도 모델 학습 실행
    print("\n" + "-"*80)
    print("📚 점진적 해상도 학습 시작")
    print("-"*80)
    
    try:
        # 모델 학습
        model, train_idx, val_idx, test_idx = progressive_resolution_training_improved(
            rgb_dir=rgb_dir,
            depth_array_dir=depth_array_dir,
            depth_image_dir=depth_image_dir,
            mde_dir=mde_dir,
            train_idx=train_idx,
            val_idx=val_idx,
            test_idx=test_idx
        )
        
        if model is None:
            raise ValueError("모델 학습이 실패했습니다.")
        
        print("✅ 모델 학습 완료!")
        
        # 최종 모델 저장
        final_model_path = 'depth_correction_model_final.h5'
        model.save(final_model_path)
        print(f"✅ 최종 모델이 저장되었습니다: {final_model_path}")
        
        # 테스트 데이터셋 생성
        print("\n테스트 데이터셋 생성 중...")
        test_ds = create_simple_dataset_with_monitoring(
            rgb_dir=rgb_dir,
            depth_array_dir=depth_array_dir,
            depth_image_dir=depth_image_dir, 
            mde_dir=mde_dir,
            indices=test_idx, 
            batch_size=8, 
            target_size=(480, 640),  # 마지막 해상도와 일치
            stage=3  # 마지막 Stage (7채널 입력)
        )
        
        if test_ds is None:
            raise ValueError("테스트 데이터셋 생성에 실패했습니다.")
        
        # 최종 모델 평가
        print("\n===== 최종 모델 평가 =====")
        test_results = model.evaluate(test_ds, verbose=1)
        print(f"테스트 손실: {test_results[0]:.4f}, MAE: {test_results[1]:.4f}")
        
        # 테스트 샘플 시각화
        metrics = evaluate_and_visualize_depth_results(model, test_ds, num_samples=5)
        
        # 종료 시간 및 총 소요 시간 계산
        end_time = time.time()
        elapsed_time = end_time - start_time
        hours, remainder = divmod(elapsed_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        
        print(f"\n⏱️ 총 학습 시간: {int(hours)}시간 {int(minutes)}분 {int(seconds)}초")
        
        return model
        
    except Exception as e:
        print(f"\n❌ 학습 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()
        print("\n⚠️ 파이프라인 실행 실패")
        return None


# 실행
if __name__ == "__main__":
    model = run_improved_depth_correction_pipeline()

✅ GPU 메모리 성장 설정 완료!

🚀 개선된 깊이 보정 파이프라인 시작
📁 visualizations 디렉토리 확인 완료
📁 checkpoints 디렉토리 확인 완료
📁 logs 디렉토리 확인 완료
📁 test_results 디렉토리 확인 완료
📊 발견된 RGB 이미지: 1008개
📊 사용할 샘플 수: 1000개
🔧 정밀도 설정: float32 (안정성 향상)

--------------------------------------------------------------------------------
📚 점진적 해상도 학습 시작
--------------------------------------------------------------------------------
학습 샘플: 600, 검증 샘플: 200, 테스트 샘플: 200

==================== 학습 단계 1/5: 해상도 (32, 32) ====================
현재 학습률: 1.00e-05
데이터셋 생성 중... 대상 해상도: (32, 32)
데이터셋 생성 중... 대상 해상도: (32, 32), Stage: 0
로드된 샘플 수: 600
데이터 전처리 시작
최종 입력 데이터 형태: (600, 32, 32, 5)
입력 데이터 범위:
  RGB: 0.0 ~ 1.0
  깊이: 0.0 ~ 1.0
  MDE: 0.0 ~ 1.0
최종 타겟 데이터 형태: (600, 32, 32, 1)
타겟 깊이 범위: 0.0 ~ 1.0
데이터셋 생성 중... 대상 해상도: (32, 32), Stage: 0
로드된 샘플 수: 200
데이터 전처리 시작
최종 입력 데이터 형태: (200, 32, 32, 5)
입력 데이터 범위:
  RGB: 0.0 ~ 1.0
  깊이: 0.0 ~ 1.0
  MDE: 0.0 ~ 1.0
최종 타겟 데이터 형태: (200, 32, 32, 1)
타겟 깊이 범위: 0.0 ~ 1.0
학습 데이터 배치 형태: 입력=(16, 32, 32, 5), 타겟=(16, 32, 32, 

/tmp/ipykernel_322296/2693354063.py:1915: RuntimeWarning: divide by zero encountered in divide
  max_ratio = np.maximum(ratio, 1/ratio)


1/1 [==============================] - 0s 83ms/step
예측 결과 형태: (2, 480, 640, 1)
1/1 [==============================] - 0s 80ms/step
예측 결과 형태: (2, 480, 640, 1)
1/1 [==============================] - 0s 82ms/step
예측 결과 형태: (2, 480, 640, 1)
1/1 [==============================] - 0s 84ms/step
예측 결과 형태: (2, 480, 640, 1)

===== 깊이 보정 성능 평가 =====
RMSE: 0.1638 ± 0.0605
MAE: 0.1468 ± 0.0638
상대 오차: 2.4792 ± 1.5981
델타(δ) < 1.25: 0.2502 ± 0.2381
델타(δ) < 1.25²: 0.6258 ± 0.1993
델타(δ) < 1.25³: 0.8645 ± 0.0851
✅ 테스트 결과가 저장되었습니다: test_results/test_metrics.txt
✅ 모델 학습 완료!
✅ 최종 모델이 저장되었습니다: depth_correction_model_final.h5

테스트 데이터셋 생성 중...
데이터셋 생성 중... 대상 해상도: (480, 640), Stage: 3
로드된 샘플 수: 200
데이터 전처리 시작
최종 입력 데이터 형태: (200, 480, 640, 7)
입력 데이터 범위:
  RGB: 0.0 ~ 1.0
  깊이: 0.0 ~ 1.0
  MDE: 0.0 ~ 1.0
  Sensor Confidence: 0.3 ~ 0.7
  MDE Confidence: 0.7 ~ 0.7
최종 타겟 데이터 형태: (200, 480, 640, 1)
타겟 깊이 범위: 0.0 ~ 1.0

===== 최종 모델 평가 =====
25/25 [==============================] - 37s 350ms/step - loss: 0.1506 - mae: